# Text Diffusion: From First Principles to Production

**The core idea**: What if instead of generating text left-to-right, we could generate all tokens at once and *iteratively refine* them? Like sculpting: start with a rough block, chip away until the statue emerges.

This is diffusion for text.


In [ ]:
# ============== SMOKE TEST SETUP ==============
# This is a minimal, fast test to validate your training loop

# 1. Create tiny synthetic dataset (no download needed)
def create_toy_dataset(n_samples=50, seq_len=32, vocab_size=50257):
    """Create a tiny random dataset for quick testing"""
    tokens = t.randint(0, vocab_size, (n_samples, seq_len))
    # Create dataset dict matching the expected format
    class ToyDataset:
        def __init__(self, tokens):
            self.tokens = tokens
        def __len__(self):
            return len(self.tokens)
        def __getitem__(self, idx):
            return {"tokens": self.tokens[idx]}
    
    return ToyDataset(tokens)

# 2. Create tiny model config
toy_model_cfg = Config(
    debug=False,
    d_model=16,      # Very small
    n_heads=4,       # Very small
    d_head=4,        # Very small
    d_mlp=16 * 2,    # Very small
    n_layers=2,      # Just 2 layers
    n_ctx=32,        # Short context
    d_vocab=reference_gpt2.cfg.d_vocab,
)

# 3. Create minimal training args
@dataclass
class ToyTrainingArgs:
    batch_size: int = 4          # Small batch
    epochs: int = 1              # Just 1 epoch
    max_steps_per_epoch: int = 5 # Only 5 steps
    lr: float = 1e-3
    weight_decay: float = 1e-2
    wandb_project: str | None = None  # Disable wandb for speed
    wandb_name: str | None = None

# 4. Complete Trainer with all methods filled in
class ToyTransformerTrainer:
    def __init__(self, args, model, train_dataset, test_dataset):
        self.model = model
        self.args = args
        self.optimizer = t.optim.AdamW(
            self.model.parameters(), lr=args.lr, weight_decay=args.weight_decay
        )
        self.step = 0
        
        # Simple dataloaders (no multiprocessing for faster startup)
        self.train_loader = DataLoader(
            train_dataset,
            batch_size=args.batch_size,
            shuffle=True,
            num_workers=0,  # No multiprocessing for speed
        )
        self.test_loader = DataLoader(
            test_dataset,
            batch_size=args.batch_size,
            shuffle=False,
            num_workers=0,
        )

    def training_step(self, batch: dict[str, Int[Tensor, "batch seq"]]) -> Float[Tensor, ""]:
        """
        Calculates the loss on the tokens in the batch, performs a gradient update step, and logs the loss.
        """
        # Extract tokens from batch
        tokens = batch["tokens"].to(device)
        
        # Forward pass
        logits = self.model(tokens)  # Shape: [batch, seq, vocab]
        
        # Compute loss (predict next token at each position)
        # Shift logits and tokens for next-token prediction
        logits_flat = logits[:, :-1, :].reshape(-1, logits.shape[-1])  # [batch*(seq-1), vocab]
        targets_flat = tokens[:, 1:].reshape(-1)  # [batch*(seq-1)]
        
        loss = t.nn.functional.cross_entropy(logits_flat, targets_flat)
        
        # Backward pass
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.step += 1
        
        # Log to wandb if enabled
        if self.args.wandb_project is not None:
            wandb.log({"train_loss": loss.item(), "step": self.step})
        
        return loss

    @t.inference_mode()
    def evaluate(self) -> float:
        """
        Evaluate the model on the test set and return the accuracy.
        """
        self.model.eval()
        
        total_correct = 0
        total_tokens = 0
        
        for batch in self.test_loader:
            tokens = batch["tokens"].to(device)
            
            # Forward pass
            logits = self.model(tokens)  # [batch, seq, vocab]
            
            # Calculate accuracy (next token prediction)
            predictions = logits[:, :-1, :].argmax(dim=-1)  # [batch, seq-1]
            targets = tokens[:, 1:]  # [batch, seq-1]
            
            correct = (predictions == targets).sum().item()
            total_correct += correct
            total_tokens += targets.numel()
        
        accuracy = total_correct / total_tokens if total_tokens > 0 else 0.0
        
        self.model.train()
        return accuracy

    def train(self):
        """
        Trains the model with minimal configuration for smoke testing.
        """
        # Initialize wandb only if project name provided
        if self.args.wandb_project is not None:
            wandb.init(
                project=self.args.wandb_project,
                name=self.args.wandb_name,
                config=vars(self.args)
            )
        
        accuracy = 0.0
        progress_bar = tqdm(total=self.args.max_steps_per_epoch * self.args.epochs)
        
        print(f"🚀 Starting smoke test training...")
        print(f"   Model params: {sum(p.numel() for p in self.model.parameters()):,}")
        
        for epoch in range(self.args.epochs):
            epoch_losses = []
            
            for i, batch in enumerate(self.train_loader):
                loss = self.training_step(batch)
                epoch_losses.append(loss.item())
                
                progress_bar.update()
                progress_bar.set_description(
                    f"Epoch {epoch + 1}, loss: {loss.item():.4f}, acc: {accuracy:.4f}"
                )
                
                if i >= self.args.max_steps_per_epoch - 1:
                    break
            
            # Evaluate after each epoch
            accuracy = self.evaluate()
            avg_loss = np.mean(epoch_losses)
            
            print(f"\n✅ Epoch {epoch + 1} complete:")
            print(f"   Avg Loss: {avg_loss:.4f}")
            print(f"   Accuracy: {accuracy:.4f}")
        
        progress_bar.close()
        
        if self.args.wandb_project is not None:
            wandb.finish()
        
        print("\n🎉 Smoke test complete! No errors detected.")
        return accuracy

# ============== RUN SMOKE TEST ==============
# Create toy datasets
print("Creating toy datasets...")
toy_train = create_toy_dataset(n_samples=50, seq_len=32)
toy_test = create_toy_dataset(n_samples=20, seq_len=32)

# Create tiny model
print("Creating tiny model...")
toy_model = DemoTransformer(toy_model_cfg).to(device)

# Create trainer
print("Creating trainer...")
toy_args = ToyTrainingArgs()
toy_trainer = ToyTransformerTrainer(toy_args, toy_model, toy_train, toy_test)

# Run training (should complete in < 10 seconds)
print("\nRunning smoke test...")
final_accuracy = toy_trainer.train()

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x42c6ce850>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Creating toy datasets...
Creating tiny model...


...

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


🚀 Starting smoke test training...
   Model params: 1,663,473


socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.



✅ Epoch 1 complete:
   Avg Loss: 10.8309
   Accuracy: 0.0000

🎉 Smoke test complete! No errors detected.
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x42c6ce850>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# | export
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
import ipywidgets as widgets
from silen_lib.utils import set_seed

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## The Problem with Left-to-Right Generation

When GPT generates text, it's stuck with a fundamental constraint: **once you write a word, you can't go back**.

Think about how *you* write. Do you write perfectly from left to right? Or do you:
- Write a rough draft
- Go back and fix awkward phrasing
- Revise the beginning after you see where the argument goes
- Polish iteratively?

Autoregressive models can't do this. They commit.


In [ ]:
# Simulating autoregressive generation
# Each token is committed forever once generated

sentence = []
tokens = ["The", "quick", "brown", "fox", "jumps"]

for i, token in enumerate(tokens):
    sentence.append(token)
    print(f"Step {i+1}: {' '.join(sentence)}")
    print(f"         Can I change 'quick' to 'slow'? {'❌ No - already committed!' if i > 1 else '✅ Not yet generated'}")
    print()


### What if we could generate differently?

**Diffusion idea**: Start with all positions as "unknown" (like `[MASK]`), then iteratively refine:


In [ ]:
# Simulating diffusion-style generation
# All positions exist simultaneously, refined over time

steps = [
    ["[MASK]", "[MASK]", "[MASK]", "[MASK]", "[MASK]"],  # Pure noise/unknown
    ["The", "[MASK]", "[MASK]", "[MASK]", "[MASK]"],     # Some clarity
    ["The", "[MASK]", "brown", "[MASK]", "[MASK]"],      # More clarity
    ["The", "quick", "brown", "[MASK]", "jumps"],        # Almost there
    ["The", "quick", "brown", "fox", "jumps"],           # Final
]

print("Diffusion-style generation (simplified):\n")
for i, step in enumerate(steps):
    print(f"Step {i}: {' '.join(step)}")
    if i < len(steps) - 1:
        print(f"        → Can still change ANY position in next step!")
    print()


### Why does this matter?

Consider this practical problem: **Fill in the blank**

> "The ___ jumped over the lazy dog"

An autoregressive model would need to generate everything up to the blank first. But what if the blank is at the beginning? Or what if you want to edit just one word in a long document?

Diffusion models can:
1. **Generate all positions in parallel** (potentially faster)
2. **Revise and refine** (like human editing)
3. **Handle infilling naturally** (the blank problem)
4. **Control generation** via guidance (steer toward desired properties)


## The Core Diffusion Intuition

Before tackling text specifically, let's understand the fundamental mechanism of diffusion. We'll use a simple 2D example that you can *see*.

**The key insight**: If you slowly corrupt data with noise, and learn to reverse each tiny corruption step, you can generate new data from pure noise.


In [ ]:
# Let's create some "data" - points that form a shape
# Imagine these are embeddings of the words "cat", "dog", "bird"

n_points = 300
theta = torch.linspace(0, 2*np.pi, n_points)

# Data forms a spiral (a recognizable pattern)
data = torch.stack([
    theta * torch.cos(theta) / 6,
    theta * torch.sin(theta) / 6
], dim=1)

plt.figure(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], alpha=0.6, s=10)
plt.title("Our 'data' - a spiral pattern")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.axis('equal')
plt.show()


### The Forward Process: Data → Noise

The forward process is simple: **gradually add Gaussian noise until the data becomes indistinguishable from random noise**.

At each step $t$, we add a little bit of noise:
$$x_t = \sqrt{1-\beta_t} \cdot x_{t-1} + \sqrt{\beta_t} \cdot \epsilon$$

where $\beta_t$ is a small noise amount and $\epsilon \sim \mathcal{N}(0, I)$

#### Where does this equation come from?

**The intuition**: We want to *interpolate* between data and noise. But we need to be careful about variance!

If we just did $x_t = (1-\beta_t) \cdot x_{t-1} + \beta_t \cdot \epsilon$, the variance would blow up over many steps.

**The math**: We want $x_t$ to have the same variance as $x_{t-1}$ (assuming $x_0$ has unit variance). For independent random variables:

$$\text{Var}(aX + bY) = a^2\text{Var}(X) + b^2\text{Var}(Y)$$

So if $\text{Var}(x_{t-1}) = 1$ and $\text{Var}(\epsilon) = 1$, we need:
$$(\sqrt{1-\beta_t})^2 \cdot 1 + (\sqrt{\beta_t})^2 \cdot 1 = (1-\beta_t) + \beta_t = 1 \checkmark$$

**That's why we use square roots!** It's not arbitrary - it preserves variance.


In [ ]:
# Visualizing variance preservation
# Why do we need those square roots?

# Wrong way: linear interpolation
wrong_x = data.clone()
wrong_variances = [wrong_x.var().item()]

# Right way: square root coefficients  
right_x = data.clone()
right_variances = [right_x.var().item()]

for t in range(num_steps):
    # WRONG: variance explodes
    wrong_noise = torch.randn_like(wrong_x)
    wrong_x = (1 - beta[t]) * wrong_x + beta[t] * wrong_noise
    wrong_variances.append(wrong_x.var().item())
    
    # RIGHT: variance stays stable
    right_noise = torch.randn_like(right_x)
    right_x = torch.sqrt(1 - beta[t]) * right_x + torch.sqrt(beta[t]) * right_noise
    right_variances.append(right_x.var().item())

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(wrong_variances, label='Wrong: linear coefficients', color='red')
plt.plot(right_variances, label='Right: sqrt coefficients', color='green')
plt.xlabel("Timestep")
plt.ylabel("Variance")
plt.title("Why sqrt? Variance preservation!")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(wrong_x[:, 0], wrong_x[:, 1], alpha=0.3, s=5, label='Wrong (variance exploded)')
plt.scatter(right_x[:, 0], right_x[:, 1], alpha=0.3, s=5, label='Right (variance preserved)')
plt.legend()
plt.title("Final distributions")
plt.axis('equal')
plt.tight_layout()
plt.show()


#### The Magic Closed-Form: Skip to Any Timestep!

Applying $x_t = \sqrt{1-\beta_t} \cdot x_{t-1} + \sqrt{\beta_t} \cdot \epsilon$ step by step is slow. Can we jump directly from $x_0$ to any $x_t$?

**Yes!** Here's the beautiful derivation:

Define $\alpha_t = 1 - \beta_t$ and $\bar{\alpha}_t = \prod_{i=1}^{t} \alpha_i$ (cumulative product).

Starting from $x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{1-\alpha_t}\epsilon_t$ and recursively substituting:

$$x_t = \sqrt{\alpha_t}(\sqrt{\alpha_{t-1}} x_{t-2} + \sqrt{1-\alpha_{t-1}}\epsilon_{t-1}) + \sqrt{1-\alpha_t}\epsilon_t$$

After all the algebra (using the fact that sum of Gaussians is Gaussian), we get:

$$\boxed{x_t = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1-\bar{\alpha}_t} \cdot \epsilon}$$

where $\epsilon \sim \mathcal{N}(0, I)$ is a **single** noise sample!

**This is powerful**: We can sample any $x_t$ directly from $x_0$ without simulating all intermediate steps.


In [ ]:
# Visualizing alpha_bar: what does it mean?

alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: alpha_bar over time
axes[0].plot(alpha_bar, linewidth=2)
axes[0].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='50/50 signal/noise')
axes[0].set_xlabel("Timestep t")
axes[0].set_ylabel("$\\bar{\\alpha}_t$")
axes[0].set_title("$\\bar{\\alpha}_t$ = how much signal remains")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: sqrt(alpha_bar) and sqrt(1-alpha_bar) - the actual coefficients
axes[1].plot(torch.sqrt(alpha_bar), label='$\\sqrt{\\bar{\\alpha}_t}$ (signal coef)', linewidth=2)
axes[1].plot(torch.sqrt(1 - alpha_bar), label='$\\sqrt{1-\\bar{\\alpha}_t}$ (noise coef)', linewidth=2)
axes[1].set_xlabel("Timestep t")
axes[1].set_ylabel("Coefficient")
axes[1].set_title("Coefficients in $x_t = \\sqrt{\\bar{\\alpha}_t}x_0 + \\sqrt{1-\\bar{\\alpha}_t}\\epsilon$")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: SNR (signal-to-noise ratio)
snr = alpha_bar / (1 - alpha_bar + 1e-8)
axes[2].plot(snr, linewidth=2)
axes[2].set_xlabel("Timestep t")
axes[2].set_ylabel("SNR (log scale)")
axes[2].set_title("Signal-to-Noise Ratio")
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At t=0:  {alpha_bar[0]:.3f} signal, {(1-alpha_bar[0]):.3f} noise → mostly data")
print(f"At t=25: {alpha_bar[25]:.3f} signal, {(1-alpha_bar[25]):.3f} noise → mixed")
print(f"At t=49: {alpha_bar[49]:.3f} signal, {(1-alpha_bar[49]):.3f} noise → mostly noise")


In [ ]:
# Forward diffusion: watch the spiral dissolve into noise

num_steps = 50
beta = torch.linspace(0.0001, 0.02, num_steps)  # Noise schedule

# Store noisy versions at different timesteps
noisy_data = [data.clone()]
x = data.clone()

for t in range(num_steps):
    noise = torch.randn_like(x)
    x = torch.sqrt(1 - beta[t]) * x + torch.sqrt(beta[t]) * noise
    noisy_data.append(x.clone())

# Visualize at key timesteps
fig, axes = plt.subplots(1, 6, figsize=(18, 3))
timesteps_to_show = [0, 10, 20, 30, 40, 50]

for ax, t in zip(axes, timesteps_to_show):
    ax.scatter(noisy_data[t][:, 0], noisy_data[t][:, 1], alpha=0.6, s=10)
    ax.set_title(f"t = {t}")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    
plt.suptitle("Forward Process: Data → Noise", y=1.02)
plt.tight_layout()
plt.show()


**Key observation**: At the end (t=50), the points look like pure Gaussian noise. The spiral structure is completely destroyed.

But here's the magic: **we can learn to reverse this!**


### The Reverse Process: Noise → Data

If we can learn to predict "which way is the data?" from any noisy point, we can reverse the process.

Think of it like this: imagine you're lost in a foggy field. If at each step you knew which direction leads toward the path, you could find your way back. That's what the neural network learns: **the direction toward clean data**.


### Score Functions: The Gradient Toward Data

Before understanding the reverse process, we need a key concept: **the score function**.

The **score** of a distribution $p(x)$ is its gradient with respect to $x$:

$$s(x) = \nabla_x \log p(x)$$

**Intuition**: The score points "uphill" toward higher probability regions. If you're in a low-density area, the score tells you which way to go to find data!

For a Gaussian $\mathcal{N}(\mu, \sigma^2)$, the score is:
$$s(x) = \nabla_x \log p(x) = -\frac{x - \mu}{\sigma^2}$$

This points toward the mean $\mu$ - exactly what we want for denoising!


In [ ]:
# Visualizing the score function for a simple 2D distribution

# Create a grid
xx, yy = torch.meshgrid(torch.linspace(-3, 3, 20), torch.linspace(-3, 3, 20), indexing='ij')
grid = torch.stack([xx.flatten(), yy.flatten()], dim=1)

# For our spiral data, the score at each point roughly points toward the nearest data point
# (This is a simplification - true score is more complex)
def estimate_score(points, data, sigma=0.5):
    """Estimate score as weighted sum of directions to data points."""
    # For each point, compute gaussian-weighted direction to all data points
    diffs = data.unsqueeze(0) - points.unsqueeze(1)  # [n_points, n_data, 2]
    dists_sq = (diffs ** 2).sum(dim=-1)  # [n_points, n_data]
    weights = torch.exp(-dists_sq / (2 * sigma**2))  # [n_points, n_data]
    weights = weights / (weights.sum(dim=1, keepdim=True) + 1e-8)
    score = (weights.unsqueeze(-1) * diffs).sum(dim=1)  # [n_points, 2]
    return score / sigma**2

scores = estimate_score(grid, data, sigma=0.3)

plt.figure(figsize=(10, 8))
plt.scatter(data[:, 0], data[:, 1], alpha=0.3, s=20, label='Data')
plt.quiver(grid[:, 0], grid[:, 1], scores[:, 0], scores[:, 1], 
           alpha=0.7, color='red', scale=50)
plt.title("Score Function: Arrows Point Toward High-Density Regions")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

print("The score function is like a vector field that pushes you toward the data!")
print("If you follow the arrows, you'll end up on the spiral.")


#### The Beautiful Connection: Score = Noise / σ²

Here's the key insight that makes diffusion work. At timestep $t$, we have:

$$x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1-\bar{\alpha}_t} \epsilon$$

Given $x_t$, what's the score $\nabla_{x_t} \log p(x_t | x_0)$?

Since $x_t | x_0 \sim \mathcal{N}(\sqrt{\bar{\alpha}_t} x_0, (1-\bar{\alpha}_t)I)$, the score is:

$$\nabla_{x_t} \log p(x_t | x_0) = -\frac{x_t - \sqrt{\bar{\alpha}_t} x_0}{1-\bar{\alpha}_t} = -\frac{\sqrt{1-\bar{\alpha}_t} \epsilon}{1-\bar{\alpha}_t} = \boxed{-\frac{\epsilon}{\sqrt{1-\bar{\alpha}_t}}}$$

**This is why predicting noise is equivalent to estimating the score!**

$$\epsilon_\theta(x_t, t) \approx \epsilon \implies s_\theta(x_t, t) \approx -\frac{\epsilon_\theta}{\sqrt{1-\bar{\alpha}_t}}$$


### Why Predict Noise? The ELBO Perspective

You might wonder: why do we predict noise $\epsilon$ instead of directly predicting $x_0$?

The answer comes from the **Evidence Lower Bound (ELBO)** - but let's build intuition first.

#### The Training Objective Derivation

We want to maximize $\log p(x_0)$ - the probability of real data under our model.

Using variational inference, we get:
$$\log p(x_0) \geq \mathbb{E}_{q}\left[ \log \frac{p(x_{0:T})}{q(x_{1:T}|x_0)} \right] = \text{ELBO}$$

After lots of math (which we'll skip), this decomposes into:
$$\text{ELBO} = \sum_{t=1}^{T} \mathbb{E}_{q(x_t|x_0)} \left[ D_{KL}(q(x_{t-1}|x_t, x_0) \| p_\theta(x_{t-1}|x_t)) \right] + \text{const}$$

**The key insight**: Both distributions are Gaussian! The KL divergence between Gaussians is:
$$D_{KL}(\mathcal{N}(\mu_1, \sigma^2) \| \mathcal{N}(\mu_2, \sigma^2)) \propto \|\mu_1 - \mu_2\|^2$$

So we're just doing **MSE between means**!


#### From MSE on Means to MSE on Noise

The true mean is (we derived this earlier):
$$\mu_{\text{true}} = \frac{1}{\sqrt{\alpha_t}}\left(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar\alpha_t}}\epsilon\right)$$

If we predict noise $\epsilon_\theta$, our predicted mean is:
$$\mu_{\theta} = \frac{1}{\sqrt{\alpha_t}}\left(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta\right)$$

The MSE between means becomes:
$$\|\mu_{\text{true}} - \mu_\theta\|^2 \propto \|\epsilon - \epsilon_\theta\|^2$$

**That's why we do MSE on noise!** It's equivalent to the theoretically justified objective, just simpler.

#### Three Equivalent Objectives

You can train by predicting:
1. **Noise $\epsilon$**: $\mathcal{L} = \|\epsilon - \epsilon_\theta(x_t, t)\|^2$ ← Most common
2. **Clean data $x_0$**: $\mathcal{L} = \|x_0 - x_{0,\theta}(x_t, t)\|^2$
3. **Score $\nabla \log p$**: $\mathcal{L} = \|s(x_t) - s_\theta(x_t, t)\|^2$

All three are mathematically equivalent (up to constants)! Noise prediction is popular because:
- Consistent magnitude across timesteps
- Works well empirically
- Easy to implement


In [ ]:
# Visualizing "which way is the data?"
# At a moderately noisy timestep, we can still see the structure

t = 20  # Moderately noisy
noisy = noisy_data[t]

# For each noisy point, the "direction toward data" is approximately 
# the vector from the noisy point toward the original point
directions = data - noisy

plt.figure(figsize=(8, 8))
plt.scatter(noisy[:, 0], noisy[:, 1], alpha=0.4, s=20, label='Noisy points')

# Show arrows for a subset of points
subset = range(0, n_points, 15)
for i in subset:
    plt.arrow(noisy[i, 0], noisy[i, 1], 
              directions[i, 0]*0.5, directions[i, 1]*0.5,
              head_width=0.08, head_length=0.04, fc='red', ec='red', alpha=0.6)

plt.title(f"At t={t}: Arrows point toward clean data")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.axis('equal')
plt.legend()
plt.show()


### Training a Denoiser: Learning the Arrows

The neural network's job: **given a noisy point and timestep t, predict the noise that was added** (equivalently, predict the direction back to clean data).

Training is simple:
1. Take clean data point $x_0$
2. Sample a random timestep $t$  
3. Add noise to get $x_t$
4. Train network to predict the noise $\epsilon$ that was added

$$\mathcal{L} = \| \epsilon - \epsilon_\theta(x_t, t) \|^2$$


In [ ]:
# A simple denoiser network for our 2D spiral
# Input: noisy point (2D) + timestep (1D) → Output: predicted noise (2D)

class SimpleDenoiser(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, hidden_dim),  # 2D point + 1D timestep
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)   # predict 2D noise
        )
    
    def forward(self, x, t):
        # t is normalized to [0, 1]
        t_normalized = t.float() / num_steps
        inputs = torch.cat([x, t_normalized.unsqueeze(-1)], dim=-1)
        return self.net(inputs)

denoiser = SimpleDenoiser().to(device)
print(f"Denoiser parameters: {sum(p.numel() for p in denoiser.parameters()):,}")


In [ ]:
# Pre-compute alpha values for efficient noising
# alpha_bar[t] = product of (1 - beta[i]) for i=0 to t
alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)

print("alpha_bar at different timesteps:")
print(f"  t=0:  {alpha_bar[0]:.4f}  (almost no noise)")
print(f"  t=25: {alpha_bar[25]:.4f} (moderate noise)")
print(f"  t=49: {alpha_bar[49]:.4f} (almost pure noise)")


In [ ]:
# | export
def forward_diffusion(x_0, t, alpha_bar):
    """
    Add noise to x_0 at timestep t in one step.
    
    x_t = sqrt(alpha_bar[t]) * x_0 + sqrt(1 - alpha_bar[t]) * noise
    
    Returns: (noisy_x, noise)
    """
    noise = torch.randn_like(x_0)
    sqrt_alpha_bar = torch.sqrt(alpha_bar[t]).view(-1, 1)
    sqrt_one_minus_alpha_bar = torch.sqrt(1 - alpha_bar[t]).view(-1, 1)
    
    x_t = sqrt_alpha_bar * x_0 + sqrt_one_minus_alpha_bar * noise
    return x_t, noise


In [ ]:
# Train the denoiser
optimizer = torch.optim.Adam(denoiser.parameters(), lr=1e-3)
data_device = data.to(device)
alpha_bar_device = alpha_bar.to(device)

losses = []
for epoch in range(2000):
    # Sample random timesteps for each data point
    t = torch.randint(0, num_steps, (n_points,), device=device)
    
    # Add noise
    x_t, noise = forward_diffusion(data_device, t, alpha_bar_device)
    
    # Predict noise
    predicted_noise = denoiser(x_t, t)
    
    # Loss: MSE between predicted and actual noise
    loss = F.mse_loss(predicted_noise, noise)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if epoch % 400 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Denoiser Training Loss")
plt.yscale('log')
plt.show()


### Generation: Running the Reverse Process

Now comes the magic. We want to reverse the forward process: go from noise back to data.

#### Deriving the Reverse Process from Bayes

The forward process is: $q(x_t | x_{t-1})$ - we know this exactly.

We want the reverse: $q(x_{t-1} | x_t)$ - but this requires knowing $p(x_0)$, which we don't!

**The key insight**: If we also condition on $x_0$, we CAN compute the reverse exactly:

$$q(x_{t-1} | x_t, x_0) = \mathcal{N}(x_{t-1}; \tilde{\mu}_t, \tilde{\beta}_t I)$$

where (after some algebra with Bayes' theorem):

$$\tilde{\mu}_t = \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1-\bar{\alpha}_t} x_0 + \frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t} x_t$$

But we don't know $x_0$! That's where the neural network comes in. We predict:

$$\hat{x}_0 = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( x_t - \sqrt{1-\bar{\alpha}_t} \epsilon_\theta(x_t, t) \right)$$

Substituting this estimate gives us:

$$\boxed{x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1-\alpha_t}{\sqrt{1-\bar\alpha_t}} \epsilon_\theta(x_t, t) \right) + \sigma_t z}$$

where $z \sim \mathcal{N}(0, I)$ adds stochasticity (except at t=0).


In [ ]:
# | export
@torch.no_grad()
def sample(model, n_samples, dim, num_steps, alpha, alpha_bar, device):
    """
    Generate samples by running the reverse diffusion process.
    """
    model.eval()
    
    # Start from pure noise
    x = torch.randn(n_samples, dim, device=device)
    trajectory = [x.cpu().clone()]
    
    for t in reversed(range(num_steps)):
        t_batch = torch.full((n_samples,), t, device=device)
        
        # Predict noise
        predicted_noise = model(x, t_batch)
        
        # Compute x_{t-1}
        alpha_t = alpha[t]
        alpha_bar_t = alpha_bar[t]
        
        # Mean of the reverse distribution
        coef1 = 1 / torch.sqrt(alpha_t)
        coef2 = (1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)
        mean = coef1 * (x - coef2 * predicted_noise)
        
        # Add noise (except at t=0)
        if t > 0:
            noise = torch.randn_like(x)
            # Variance is beta_t (simplified)
            sigma = torch.sqrt(1 - alpha_t)
            x = mean + sigma * noise
        else:
            x = mean
            
        trajectory.append(x.cpu().clone())
    
    return x, trajectory


In [ ]:
# Generate new samples!
generated, trajectory = sample(denoiser, 300, 2, num_steps, alpha.to(device), alpha_bar.to(device), device)
generated = generated.cpu()

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(data[:, 0], data[:, 1], alpha=0.6, s=10, label='Original data')
plt.title("Original Spiral")
plt.axis('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.subplot(1, 2, 2)
plt.scatter(generated[:, 0], generated[:, 1], alpha=0.6, s=10, c='orange', label='Generated')
plt.title("Generated from Pure Noise!")
plt.axis('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.tight_layout()
plt.show()


In [ ]:
# Watch the generation process unfold
fig, axes = plt.subplots(1, 6, figsize=(18, 3))
steps_to_show = [0, 10, 25, 40, 49, 50]  # 0 is pure noise, 50 is final

for ax, s in zip(axes, steps_to_show):
    ax.scatter(trajectory[s][:, 0], trajectory[s][:, 1], alpha=0.6, s=10)
    ax.set_title(f"Step {s}" if s < 50 else "Final")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')

plt.suptitle("Reverse Process: Noise → Data (Generation)", y=1.02)
plt.tight_layout()
plt.show()


### Summary: The Diffusion Recipe

1. **Forward process**: Gradually add noise until data → pure noise
2. **Training**: Teach a network to predict the noise at each step
3. **Generation**: Start from noise, iteratively remove predicted noise

That's it. That's diffusion. 

### Bonus: Faster Sampling with DDIM

Our 50-step sampling is slow. **DDIM** (Denoising Diffusion Implicit Models) can speed this up!

**The insight**: DDPM adds stochasticity at each step. But if we remove it, we get a **deterministic** mapping from noise to data - and we can skip steps!

| Method | Steps Needed | Stochastic? | Quality |
|--------|--------------|-------------|---------|
| DDPM | All T steps | Yes | Best |
| DDIM | 10-50 steps | No | Nearly as good |

DDIM modifies the reverse step to:
$$x_{t-1} = \sqrt{\bar\alpha_{t-1}} \hat{x}_0 + \sqrt{1-\bar\alpha_{t-1}} \cdot \frac{x_t - \sqrt{\bar\alpha_t}\hat{x}_0}{\sqrt{1-\bar\alpha_t}}$$

No randomness added → can skip timesteps → 5-10x faster sampling!

**Now the question**: How do we apply this to *text*?


## The Text Problem: Discrete vs Continuous

Here's the fundamental challenge: **text is discrete**.

In images, a pixel can smoothly transition: `[0.5, 0.5, 0.5]` → `[0.51, 0.49, 0.52]`

In text, tokens are integers: "cat" is token `3784`. What's `3784 + 0.3`?


In [ ]:
# The discrete problem illustrated

vocab = {"cat": 0, "dog": 1, "bird": 2, "fish": 3}

# This makes sense for images
pixel = torch.tensor([0.5, 0.5, 0.5])
noisy_pixel = pixel + 0.1 * torch.randn(3)
print(f"Image pixel:       {pixel}")
print(f"Noisy image pixel: {noisy_pixel}")
print(f"Still valid RGB? ✅ Yes\n")

# This doesn't make sense for tokens
token_id = vocab["cat"]  # = 0
print(f"Token 'cat' = {token_id}")
print(f"'cat' + noise = {token_id + 0.3:.1f}")
print(f"What word is 0.3? ❌ Nonsense!")


In [ ]:
# Visualizing continuous vs discrete: why text is harder

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Image: continuous space
ax = axes[0]
# Show color gradient
gradient = np.linspace(0, 1, 100).reshape(1, -1)
gradient = np.repeat(gradient, 20, axis=0)
ax.imshow(gradient, aspect='auto', cmap='RdYlGn')
ax.set_title("Image: Continuous\n(infinite values between red and green)", fontsize=11)
ax.set_xticks([0, 50, 99])
ax.set_xticklabels(['Red\n(0.0)', 'Yellow\n(0.5)', 'Green\n(1.0)'])
ax.set_yticks([])
ax.annotate("", xy=(60, 10), xytext=(40, 10), 
            arrowprops=dict(arrowstyle="<->", lw=2))
ax.text(50, 15, "can smoothly\ninterpolate", ha='center', fontsize=9)

# Text: discrete space
ax = axes[1]
words = ['cat', 'dog', 'bird', 'fish']
for i, w in enumerate(words):
    ax.scatter(i, 0, s=300, c='blue', zorder=5)
    ax.annotate(w, (i, 0.1), ha='center', fontsize=11)
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(-0.5, 0.5)
ax.set_title("Text: Discrete\n(only 4 valid values!)", fontsize=11)
ax.axhline(0, color='gray', linestyle='--', alpha=0.3)
ax.scatter(0.5, 0, s=100, c='red', marker='x', linewidths=2, zorder=5)
ax.annotate("??? (invalid!)", (0.5, -0.2), ha='center', fontsize=9, color='red')
ax.set_yticks([])
ax.set_xticks([])

# The two solutions
ax = axes[2]
ax.text(0.5, 0.9, "Two Solutions", fontsize=14, ha='center', fontweight='bold', transform=ax.transAxes)
ax.text(0.5, 0.7, "1️⃣ Embedding Space", fontsize=12, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.55, "Map tokens → continuous vectors\nDiffuse in vector space\nRound back to tokens", fontsize=10, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.35, "2️⃣ Discrete Diffusion", fontsize=12, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.2, "Replace 'Gaussian noise' with\n'random token substitution'\nor 'masking'", fontsize=10, ha='center', transform=ax.transAxes)
ax.axis('off')

plt.tight_layout()
plt.show()


### Two Solutions to the Discrete Problem

**Approach 1: Embedding Space Diffusion**
- Map tokens to continuous embeddings
- Diffuse in embedding space
- Map back to tokens

**Approach 2: Discrete Diffusion**  
- Don't use Gaussian noise at all
- Instead, *corrupt* by randomly replacing tokens
- Often with a special `[MASK]` token

Let's explore both.


## Approach 1: Embedding Space Diffusion

The idea: tokens live in discrete space, but their **embeddings** are continuous vectors. We can diffuse there!


In [ ]:
# Visualizing embedding space
# Each word is a point in continuous space

# Pretend embeddings for words (2D for visualization)
word_embeddings = {
    "cat": torch.tensor([1.0, 0.5]),
    "dog": torch.tensor([1.2, 0.3]),
    "bird": torch.tensor([-0.5, 1.0]),
    "fish": torch.tensor([-0.3, -1.0]),
    "happy": torch.tensor([2.0, 2.0]),
    "sad": torch.tensor([2.2, -2.0]),
}

plt.figure(figsize=(8, 6))
for word, emb in word_embeddings.items():
    plt.scatter(emb[0], emb[1], s=100)
    plt.annotate(word, (emb[0]+0.1, emb[1]+0.1), fontsize=12)

plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.xlabel("Embedding Dimension 1")
plt.ylabel("Embedding Dimension 2")
plt.title("Words as Points in Continuous Embedding Space")
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Now we CAN add noise to embeddings!
cat_emb = word_embeddings["cat"]
noisy_cat = cat_emb + 0.3 * torch.randn(2)

print(f"'cat' embedding: {cat_emb}")
print(f"Noisy 'cat':     {noisy_cat}")
print(f"\nThis is valid! It's a point in continuous space.")


### The Rounding Problem

After diffusing, we need to get back to actual tokens. But what if our denoised embedding doesn't land exactly on a word?


In [ ]:
# The rounding problem: what word is closest?

def nearest_word(point, embeddings):
    """Find the nearest word to a point in embedding space."""
    min_dist = float('inf')
    nearest = None
    for word, emb in embeddings.items():
        dist = torch.norm(point - emb)
        if dist < min_dist:
            min_dist = dist
            nearest = word
    return nearest, min_dist

# Suppose after denoising we land here:
denoised_point = torch.tensor([0.8, 0.9])

nearest, dist = nearest_word(denoised_point, word_embeddings)
print(f"Denoised embedding: {denoised_point}")
print(f"Nearest word: '{nearest}' (distance: {dist:.2f})")

# Visualize
plt.figure(figsize=(8, 6))
for word, emb in word_embeddings.items():
    plt.scatter(emb[0], emb[1], s=100, label=word)
    plt.annotate(word, (emb[0]+0.1, emb[1]+0.1), fontsize=10)

plt.scatter(denoised_point[0], denoised_point[1], s=200, c='red', marker='x', linewidths=3, label='Denoised point')
plt.plot([denoised_point[0], word_embeddings[nearest][0]], 
         [denoised_point[1], word_embeddings[nearest][1]], 'r--', alpha=0.5)
plt.title(f"Rounding: denoised point → nearest word '{nearest}'")
plt.grid(True, alpha=0.3)
plt.show()


### Embedding Space Diffusion: Pros and Cons

**Pros:**
- We can reuse all the continuous diffusion machinery
- Smooth gradients, well-understood math
- This is what **Diffusion-LM** does (we'll explore it later)

**Cons:**
- Rounding errors accumulate
- Generated embeddings might not match any real word well
- Needs careful training to keep embeddings "on manifold"

This is a valid approach and powers some influential models. But there's another way...


## Approach 2: Discrete Diffusion (The Modern Approach)

What if we redefine "noise" entirely for discrete data?

Instead of Gaussian noise, we use **token corruption**:
- Replace tokens with `[MASK]` (absorbing diffusion)
- Replace tokens with random tokens (uniform diffusion)

The forward process becomes: gradually corrupt tokens until they're all `[MASK]` or random garbage.

The reverse process: predict the original tokens from the corrupted ones.


In [ ]:
# Discrete "noise" = token replacement

sentence = ["The", "cat", "sat", "on", "the", "mat"]
MASK = "[MASK]"

# Forward process: gradually replace tokens with [MASK]
def corrupt_sentence(tokens, mask_prob):
    """Replace each token with [MASK] with probability mask_prob."""
    corrupted = []
    for token in tokens:
        if torch.rand(1).item() < mask_prob:
            corrupted.append(MASK)
        else:
            corrupted.append(token)
    return corrupted

print("Forward process (discrete):\n")
for prob in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    corrupted = corrupt_sentence(sentence, prob)
    print(f"mask_prob={prob:.1f}: {' '.join(corrupted)}")


### Connection to BERT: MLM is One-Step Diffusion!

If you know BERT, you already understand discrete diffusion:
- BERT: mask 15% of tokens, predict them → **one step** of denoising
- Discrete diffusion: mask variable %, predict them → **many steps** of denoising

The key insight: diffusion is just BERT's masked language modeling, done *iteratively* with a *schedule*.


### Transition Matrices: Formalizing Discrete Noise

In continuous diffusion, we had: $x_t = \sqrt{1-\beta_t} x_{t-1} + \sqrt{\beta_t} \epsilon$

In discrete diffusion, we use a **transition matrix** $Q_t$:
- $Q_t[i,j]$ = probability of token $i$ becoming token $j$ at step $t$

For absorbing (mask-based) diffusion with vocabulary of size $V$ plus `[MASK]`:

$$Q_t = \begin{bmatrix} 1-\beta_t & 0 & \cdots & \beta_t \\ 0 & 1-\beta_t & \cdots & \beta_t \\ \vdots & & \ddots & \vdots \\ 0 & 0 & \cdots & 1 \end{bmatrix}$$

The last column is the `[MASK]` token - once you're masked, you stay masked (absorbing state).


In [ ]:
# | export
def make_absorbing_transition_matrix(vocab_size, beta):
    """
    Create absorbing transition matrix.
    vocab_size includes the MASK token as the last index.
    """
    Q = torch.zeros(vocab_size, vocab_size)
    
    # Regular tokens: stay with prob (1-beta), go to MASK with prob beta
    for i in range(vocab_size - 1):
        Q[i, i] = 1 - beta
        Q[i, vocab_size - 1] = beta
    
    # MASK token: always stays as MASK (absorbing)
    Q[vocab_size - 1, vocab_size - 1] = 1.0
    
    return Q


In [ ]:
# Visualize a small transition matrix
# Vocab: [cat, dog, bird, fish, MASK]
small_vocab = ["cat", "dog", "bird", "fish", "[MASK]"]
Q = make_absorbing_transition_matrix(5, beta=0.3)

plt.figure(figsize=(6, 5))
plt.imshow(Q, cmap='Blues', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.xticks(range(5), small_vocab, rotation=45)
plt.yticks(range(5), small_vocab)
plt.xlabel("To token")
plt.ylabel("From token")
plt.title("Absorbing Transition Matrix (β=0.3)")

# Add probability values
for i in range(5):
    for j in range(5):
        if Q[i,j] > 0.01:
            plt.text(j, i, f'{Q[i,j]:.1f}', ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Multiple steps compound: Q_bar = Q_1 @ Q_2 @ ... @ Q_t
# After many steps, everything ends up as [MASK]

Q_bar = torch.eye(5)
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
steps_to_show = [1, 5, 10, 20]

for ax, num_steps in zip(axes, steps_to_show):
    Q_bar = torch.eye(5)
    for _ in range(num_steps):
        Q_bar = Q_bar @ Q
    
    ax.imshow(Q_bar, cmap='Blues', vmin=0, vmax=1)
    ax.set_xticks(range(5))
    ax.set_xticklabels(small_vocab, rotation=45, fontsize=8)
    ax.set_yticks(range(5))
    ax.set_yticklabels(small_vocab, fontsize=8)
    ax.set_title(f"After {num_steps} steps")

plt.suptitle("Tokens gradually flow to [MASK]", y=1.02)
plt.tight_layout()
plt.show()


### Alternative: Uniform Diffusion

Instead of tokens → `[MASK]`, tokens can become *any* token:

$$Q_t[i,j] = \begin{cases} 1 - \beta_t & \text{if } i = j \\ \beta_t / V & \text{otherwise} \end{cases}$$

This is like adding "discrete noise" - tokens randomly flip to other tokens.


In [ ]:
# | export
def make_uniform_transition_matrix(vocab_size, beta):
    """
    Create uniform transition matrix.
    Each token can become any other token with equal probability.
    """
    # Off-diagonal: beta / (V-1)
    Q = torch.full((vocab_size, vocab_size), beta / (vocab_size - 1))
    # Diagonal: 1 - beta
    Q.fill_diagonal_(1 - beta)
    return Q

Q_uniform = make_uniform_transition_matrix(4, beta=0.3)  # Just 4 regular tokens

plt.figure(figsize=(5, 4))
plt.imshow(Q_uniform, cmap='Blues', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.xticks(range(4), ["cat", "dog", "bird", "fish"])
plt.yticks(range(4), ["cat", "dog", "bird", "fish"])
plt.xlabel("To token")
plt.ylabel("From token")
plt.title("Uniform Transition Matrix (β=0.3)")

for i in range(4):
    for j in range(4):
        plt.text(j, i, f'{Q_uniform[i,j]:.2f}', ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.show()


### Absorbing vs Uniform: Which to Use?

| Approach | Forward Process | Pros | Cons |
|----------|-----------------|------|------|
| **Absorbing** | tokens → [MASK] | Simpler, fewer possibilities | Needs [MASK] token |
| **Uniform** | tokens → any token | More gradual corruption | Harder to learn |

**In practice**: Absorbing (mask-based) is more common and easier to train. We'll use it.

---

## Building a Discrete Diffusion Model

Now let's build a working discrete diffusion model for text. We'll train it on a simple pattern to see it learn.


In [ ]:
# First, let's create a simple dataset
# We'll learn to generate short "sentences" with structure

# Simple vocabulary
VOCAB = ["the", "a", "cat", "dog", "bird", "sat", "ran", "flew", "on", "in", "[MASK]", "[PAD]"]
VOCAB_SIZE = len(VOCAB)
MASK_ID = VOCAB.index("[MASK]")
PAD_ID = VOCAB.index("[PAD]")

# Create word-to-id and id-to-word mappings
word2id = {w: i for i, w in enumerate(VOCAB)}
id2word = {i: w for w, i in word2id.items()}

print("Vocabulary:")
for i, w in enumerate(VOCAB):
    print(f"  {i}: '{w}'")


In [ ]:
# Generate training data: simple structured sentences
# Pattern: [article] [animal] [verb] [preposition] [article] [animal]

def generate_sentence():
    """Generate a random structured sentence as token ids."""
    articles = ["the", "a"]
    animals = ["cat", "dog", "bird"]
    verbs = ["sat", "ran", "flew"]
    preps = ["on", "in"]
    
    words = [
        np.random.choice(articles),
        np.random.choice(animals),
        np.random.choice(verbs),
        np.random.choice(preps),
        np.random.choice(articles),
        np.random.choice(animals),
    ]
    return torch.tensor([word2id[w] for w in words])

# Generate some examples
print("Sample training sentences:")
for _ in range(5):
    ids = generate_sentence()
    words = [id2word[i.item()] for i in ids]
    print(f"  {' '.join(words)}")


In [ ]:
# Create a training dataset
SEQ_LEN = 6  # Our sentences are 6 tokens
train_data = torch.stack([generate_sentence() for _ in range(1000)])

print(f"Training data shape: {train_data.shape}")
print(f"(1000 sentences, each 6 tokens)")


### The Noise Schedule for Discrete Diffusion

Just like continuous diffusion, we need a schedule that controls how fast tokens get masked.

We'll use a **cosine schedule** - it's smoother than linear and works better in practice.


In [ ]:
# | export
def cosine_schedule(num_steps, s=0.008):
    """
    Cosine noise schedule.
    Returns alpha_bar (cumulative product of 1-beta) at each timestep.
    """
    t = torch.linspace(0, num_steps, num_steps + 1)
    f_t = torch.cos((t / num_steps + s) / (1 + s) * np.pi / 2) ** 2
    alpha_bar = f_t / f_t[0]
    return alpha_bar[1:]  # Remove t=0

NUM_DIFFUSION_STEPS = 100
alpha_bar_discrete = cosine_schedule(NUM_DIFFUSION_STEPS)

plt.figure(figsize=(10, 4))
plt.plot(alpha_bar_discrete, label='ᾱ (prob of keeping original)')
plt.plot(1 - alpha_bar_discrete, label='1-ᾱ (prob of being masked)', linestyle='--')
plt.xlabel("Timestep t")
plt.ylabel("Probability")
plt.title("Cosine Noise Schedule")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"At t=0: {alpha_bar_discrete[0]:.3f} chance of keeping original (barely noisy)")
print(f"At t=50: {alpha_bar_discrete[50]:.3f} chance of keeping original")
print(f"At t=99: {alpha_bar_discrete[99]:.3f} chance of keeping original (almost all masked)")


In [ ]:
# | export
def discrete_forward_diffusion(x_0, t, alpha_bar, mask_id):
    """
    Corrupt tokens by randomly replacing with [MASK].
    
    Args:
        x_0: Original tokens [batch_size, seq_len]
        t: Timestep for each sample [batch_size]
        alpha_bar: Noise schedule [num_steps]
        mask_id: ID of the [MASK] token
    
    Returns:
        x_t: Corrupted tokens [batch_size, seq_len]
    """
    # Get masking probability for each sample
    mask_prob = 1 - alpha_bar[t]  # [batch_size]
    
    # Sample mask: which positions to corrupt
    mask = torch.rand_like(x_0.float()) < mask_prob.unsqueeze(-1)
    
    # Apply mask
    x_t = torch.where(mask, torch.full_like(x_0, mask_id), x_0)
    
    return x_t, mask


In [ ]:
# Demonstrate discrete forward diffusion on a single sentence
example = train_data[0:1]  # [1, 6]
print("Original:", ' '.join([id2word[i.item()] for i in example[0]]))
print()

for t_val in [0, 25, 50, 75, 99]:
    t = torch.tensor([t_val])
    corrupted, _ = discrete_forward_diffusion(example, t, alpha_bar_discrete, MASK_ID)
    words = [id2word[i.item()] for i in corrupted[0]]
    print(f"t={t_val:2d}: {' '.join(words)}")


### The Denoising Network

The network takes:
- Corrupted tokens $x_t$
- Timestep $t$

And predicts: **logits for each token position** (what should the original token be?)

We'll use a simple Transformer - it needs to see all positions to make good predictions.


In [ ]:
# | export
class SinusoidalPosEmb(nn.Module):
    """Sinusoidal position embedding for timesteps (from original diffusion paper)."""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = np.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

# Demo: timestep embeddings are different for different t
time_emb = SinusoidalPosEmb(32)
t_examples = torch.tensor([0, 25, 50, 75, 99], dtype=torch.float)
embeddings = time_emb(t_examples)

plt.figure(figsize=(12, 3))
plt.imshow(embeddings.detach().numpy(), aspect='auto', cmap='RdBu')
plt.colorbar()
plt.xlabel("Embedding dimension")
plt.ylabel("Timestep")
plt.yticks(range(5), ['t=0', 't=25', 't=50', 't=75', 't=99'])
plt.title("Sinusoidal Timestep Embeddings")
plt.show()


In [ ]:
# | export
class DiscreteTextDenoiser(nn.Module):
    """
    A simple Transformer-based denoiser for discrete text diffusion.
    """
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=2, max_seq_len=64):
        super().__init__()
        self.d_model = d_model
        
        # Token embedding
        self.token_emb = nn.Embedding(vocab_size, d_model)
        
        # Position embedding
        self.pos_emb = nn.Embedding(max_seq_len, d_model)
        
        # Timestep embedding
        self.time_emb = nn.Sequential(
            SinusoidalPosEmb(d_model),
            nn.Linear(d_model, d_model * 2),
            nn.GELU(),
            nn.Linear(d_model * 2, d_model),
        )
        
        # Transformer encoder (bidirectional - can see all positions)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=d_model * 4,
            batch_first=True,
            norm_first=True,  # Pre-norm for stability
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output projection to vocabulary
        self.output_proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, x, t):
        """
        Args:
            x: Token ids [batch_size, seq_len]
            t: Timesteps [batch_size]
        Returns:
            logits: [batch_size, seq_len, vocab_size]
        """
        batch_size, seq_len = x.shape
        
        # Token + position embeddings
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)
        h = self.token_emb(x) + self.pos_emb(positions)
        
        # Add timestep embedding to all positions
        t_emb = self.time_emb(t.float())  # [batch_size, d_model]
        h = h + t_emb.unsqueeze(1)
        
        # Transformer
        h = self.transformer(h)
        
        # Project to vocabulary
        logits = self.output_proj(h)
        
        return logits


In [ ]:
# Create the model
model = DiscreteTextDenoiser(
    vocab_size=VOCAB_SIZE,
    d_model=128,
    nhead=4,
    num_layers=2,
    max_seq_len=SEQ_LEN
).to(device)

n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")

# Quick test
test_x = train_data[:2].to(device)
test_t = torch.tensor([10, 50], device=device)
test_output = model(test_x, test_t)
print(f"Input shape: {test_x.shape}")
print(f"Output shape: {test_output.shape}  (batch, seq_len, vocab_size)")


### Training: The Denoising Loss

The loss is simple: **cross-entropy on the masked positions only**.

We don't care about predicting tokens that weren't masked - we only train the model to "fill in the blanks".


In [ ]:
# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
train_data_device = train_data.to(device)
alpha_bar_device = alpha_bar_discrete.to(device)

batch_size = 64
losses = []

for epoch in range(500):
    model.train()
    
    # Sample batch
    idx = torch.randint(0, len(train_data), (batch_size,))
    x_0 = train_data_device[idx]
    
    # Sample random timesteps
    t = torch.randint(0, NUM_DIFFUSION_STEPS, (batch_size,), device=device)
    
    # Forward diffusion (add noise)
    x_t, mask = discrete_forward_diffusion(x_0, t, alpha_bar_device, MASK_ID)
    
    # Predict logits
    logits = model(x_t, t)
    
    # Loss: cross-entropy on masked positions only
    # We use the original tokens as targets
    loss = F.cross_entropy(
        logits.view(-1, VOCAB_SIZE),
        x_0.view(-1),
        reduction='none'
    )
    
    # Only count loss on masked positions
    loss = (loss.view(batch_size, -1) * mask.float()).sum() / mask.float().sum()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Discrete Text Diffusion Training Loss")
plt.grid(True, alpha=0.3)
plt.show()


### Generation: Iterative Unmasking

To generate, we reverse the process:
1. Start with all `[MASK]` tokens
2. At each step, predict tokens for masked positions
3. Unmask some positions based on confidence
4. Repeat until all unmasked

This is like BERT demasking, but done iteratively!


In [ ]:
# | export
@torch.no_grad()
def sample_discrete(model, seq_len, num_steps, alpha_bar, mask_id, vocab_size, device, n_samples=1):
    """
    Generate text by iteratively unmasking.
    """
    model.eval()
    
    # Start with all masks
    x = torch.full((n_samples, seq_len), mask_id, device=device)
    trajectory = [x.cpu().clone()]
    
    for t in reversed(range(num_steps)):
        t_batch = torch.full((n_samples,), t, device=device)
        
        # Predict logits for all positions
        logits = model(x, t_batch)
        
        # Sample from predicted distribution (with temperature)
        probs = F.softmax(logits, dim=-1)
        predicted = torch.multinomial(probs.view(-1, vocab_size), 1).view(n_samples, seq_len)
        
        # Determine which positions should be unmasked at this step
        # Use the schedule: prob of being unmasked = 1 - alpha_bar[t] going to 1 - alpha_bar[t-1]
        if t > 0:
            mask_prob_now = 1 - alpha_bar[t]
            mask_prob_prev = 1 - alpha_bar[t-1]
            # Unmask with probability (mask_prob_now - mask_prob_prev) / mask_prob_now
            unmask_prob = (mask_prob_now - mask_prob_prev) / (mask_prob_now + 1e-8)
            unmask = torch.rand_like(x.float()) < unmask_prob
        else:
            # Last step: unmask everything remaining
            unmask = (x == mask_id)
        
        # Only unmask positions that are currently masked
        unmask = unmask & (x == mask_id)
        
        # Apply predictions to unmasked positions
        x = torch.where(unmask, predicted, x)
        trajectory.append(x.cpu().clone())
    
    return x, trajectory


In [ ]:
# Generate some samples!
generated, gen_trajectory = sample_discrete(
    model, SEQ_LEN, NUM_DIFFUSION_STEPS, alpha_bar_device, MASK_ID, VOCAB_SIZE, device, n_samples=10
)

print("Generated sentences:\n")
for i in range(10):
    words = [id2word[tok.item()] for tok in generated[i]]
    print(f"  {' '.join(words)}")


In [ ]:
# Visualize the generation process for one sample
print("Generation process (one sample):\n")

steps_to_show = [0, 25, 50, 75, 90, 100]
for step in steps_to_show:
    words = [id2word[tok.item()] for tok in gen_trajectory[step][0]]
    n_masked = sum(1 for w in words if w == "[MASK]")
    print(f"Step {step:3d}: {' '.join(words):50s}  ({n_masked} masks remaining)")


### Test Problem: Confidence-Based Sampling

Currently our sampler unmasks positions randomly. A better strategy: **unmask the most confident predictions first**.

Fill in the function below to implement confidence-based sampling:


In [ ]:
def test_confidence_based_unmask(logits, x, mask_id, n_to_unmask):
    """
    Given logits and current tokens, return which positions to unmask.
    
    Args:
        logits: [batch_size, seq_len, vocab_size]
        x: Current token ids [batch_size, seq_len]  
        mask_id: ID of [MASK] token
        n_to_unmask: How many positions to unmask this step
        
    Returns:
        unmask: Boolean tensor [batch_size, seq_len] of positions to unmask
    """
    batch_size, seq_len, vocab_size = logits.shape
    
    # Get confidence (max probability) for each position
    test_probs = F.softmax(logits, dim=-1)
    test_confidence = test_probs.max(dim=-1).values  # [batch_size, seq_len]
    
    # Only consider masked positions
    test_is_masked = (x == mask_id)
    
    # FILL IN: Set confidence of non-masked positions to -inf so they won't be selected
    # test_confidence = ...
    
    # FILL IN: For each sample, find the top-k most confident masked positions
    # Hint: use torch.topk
    # test_top_indices = ...
    
    # FILL IN: Create boolean mask for positions to unmask
    # test_unmask = ...
    
    return test_unmask

# Test your implementation:
# If correct, this should unmask the 2 most confident masked positions
test_logits = torch.randn(1, 6, VOCAB_SIZE)
test_x = torch.tensor([[MASK_ID, 2, MASK_ID, MASK_ID, 3, MASK_ID]])  # 4 masked positions
# test_result = test_confidence_based_unmask(test_logits, test_x, MASK_ID, n_to_unmask=2)
# assert test_result.sum() == 2, "Should unmask exactly 2 positions"
# print("Test passed!")


---

## Connecting the Dots: Diffusion as Denoising Autoencoders

Now that we've built and trained a working model, let's connect to a fundamental insight:

> **"A diffusion model is basically a denoising autoencoder trained in a very particular way, on a very extreme noise schedule, with a Markov chain structure."**

Let's unpack this.


### What's a Denoising Autoencoder (DAE)?

You know regular autoencoders: encode → bottleneck → decode to reconstruct input.

A **Denoising Autoencoder** has one twist: 
- Input: **corrupted** data (add noise, mask parts, etc.)
- Target: **original** clean data
- The network learns to "denoise"

Sound familiar? That's exactly what we just built!


In [ ]:
### DAE vs Diffusion: Side-by-Side Comparison

| Aspect | **Denoising Autoencoder** | **Diffusion Model** |
|--------|--------------------------|---------------------|
| **Training** | 1. Take clean data $x$ | 1. Take clean data $x_0$ |
| | 2. Corrupt once: $\tilde{x} = \text{corrupt}(x)$ | 2. Corrupt at MANY levels: $x_t = \text{corrupt}(x_0, t)$ |
| | 3. Train: $f(\tilde{x}) \to x$ | 3. Train: $f(x_t, t) \to \epsilon$ or $x_0$ |
| **Corruption** | Single fixed level | Continuous spectrum from barely noisy → pure noise |
| **Network awareness** | Doesn't know noise level | Receives timestep $t$ as input |
| **Generation** | ❌ Can't generate - only reconstructs | ✅ Can generate from pure noise! |

The key difference: **Diffusion learns to denoise from ALL noise levels**, including pure noise. This is what enables generation!


### The Three Key Differences

**1. "Very particular way"** - Training on ALL noise levels
- DAE: one fixed corruption level
- Diffusion: trained on every noise level from barely noisy to pure noise

**2. "Extreme noise schedule"** - We go all the way to pure noise
- DAE: mild corruption (you can still see the original)
- Diffusion: at t=T, data is completely destroyed

**3. "Markov chain structure"** - Each step only depends on the previous
- This lets us decompose the hard problem (noise → data) into many easy steps
- Each step only needs to remove a tiny bit of noise


In [ ]:
# Visualizing the Markov chain insight: breaking hard into easy

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Left: The impossible single-step problem
ax = axes[0]
ax.text(0.1, 0.8, "Pure Noise", fontsize=14, fontweight='bold')
ax.text(0.1, 0.6, "[MASK] [MASK] [MASK] [MASK] [MASK] [MASK]", fontsize=10, family='monospace')
ax.annotate("", xy=(0.9, 0.5), xytext=(0.1, 0.5),
            arrowprops=dict(arrowstyle="->", lw=3, color='red'))
ax.text(0.5, 0.52, "???", fontsize=16, ha='center', color='red', fontweight='bold')
ax.text(0.7, 0.8, "Clean Data", fontsize=14, fontweight='bold')
ax.text(0.55, 0.6, "the cat sat on the mat", fontsize=10, family='monospace')
ax.text(0.5, 0.2, "ONE GIANT LEAP\n(Impossible to learn!)", fontsize=12, ha='center', color='red')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title("❌ The Hard Way: One Step", fontsize=14)

# Right: The easy multi-step problem
ax = axes[1]
steps = [
    "[M] [M] [M] [M] [M] [M]",
    "[M] [M] [M] [M] [M] mat",
    "[M] [M] sat [M] [M] mat",
    "the [M] sat [M] the mat",
    "the cat sat on the mat",
]
for i, step in enumerate(steps):
    y = 0.85 - i * 0.17
    ax.text(0.1, y, step, fontsize=9, family='monospace')
    if i < len(steps) - 1:
        ax.annotate("", xy=(0.08, y - 0.05), xytext=(0.08, y - 0.12),
                    arrowprops=dict(arrowstyle="->", lw=1.5, color='green'))
ax.text(0.5, 0.05, "MANY SMALL STEPS\n(Each step is easy!)", fontsize=12, ha='center', color='green')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title("✅ The Easy Way: Many Steps (Markov Chain)", fontsize=14)

plt.tight_layout()
plt.show()


---

## Deep Dive: Production Text Diffusion Models

Now let's look at real models. We'll progress through the evolution:

1. **Diffusion-LM** (2022) - The foundational paper, continuous embeddings
2. **DiffuSeq** (2023) - Seq2seq conditioning
3. **MMDiT** (2024) - Modern transformer-based diffusion

Let's start by loading Diffusion-LM and understanding its architecture.

<!-- 
IMAGE REQUEST: Diffusion-LM architecture diagram
URL: https://arxiv.org/abs/2205.14217 (Figure 1)
Description: Shows the embedding diffusion process with encoder and decoder
-->


### Diffusion-LM: The Foundation (Deep Dive)

**Paper**: "Diffusion-LM Improves Controllable Text Generation" (Li et al., 2022)

#### Why Diffusion-LM Matters

Diffusion-LM was the first to show diffusion could work for text generation. It solved the discrete→continuous problem elegantly and introduced key ideas that every subsequent model builds on.

**What you'll learn here:**
1. How to diffuse in embedding space (the continuous trick)
2. Score matching for text - what does the score mean for embeddings?
3. The rounding problem and how to solve it
4. Classifier guidance - steering generation toward desired attributes
5. Noise schedules tuned for text
6. Sampling tradeoffs (steps vs quality)

#### The Core Architecture

```
Text: "The cat sat"
    ↓ Embedding Layer (LEARNED, not frozen!)
Embeddings: [[0.2, -0.1, ...], [0.5, 0.3, ...], [0.1, 0.8, ...]]  
    ↓ Add noise (standard Gaussian diffusion)
Noisy Embeddings: [[0.3, 0.1, ...], [0.2, 0.5, ...], [0.4, 0.6, ...]]
    ↓ Transformer Denoiser (with timestep conditioning)  
Denoised Embeddings: [[0.19, -0.08, ...], [0.51, 0.29, ...], [0.12, 0.81, ...]]
    ↓ Rounding (project to nearest token embedding)
Text: "The cat sat"
```

#### Why Learned Embeddings?

If we used frozen embeddings (like Word2Vec or frozen BERT):
- Embeddings are clustered in complex manifolds
- Diffusion would push points off the manifold
- Hard to round back to valid tokens

With **learned embeddings**:
- The model can shape the embedding space for diffusion
- Embeddings spread out to make denoising easier
- Trained end-to-end with the diffusion objective


In [ ]:
# Visualizing embedding space diffusion vs discrete diffusion

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Left: Token space (discrete)
ax = axes[0]
tokens = ['cat', 'dog', 'bird', 'fish', 'run', 'fly']
for i, tok in enumerate(tokens):
    ax.scatter(i, 0, s=200, c='blue')
    ax.annotate(tok, (i, 0.05), ha='center', fontsize=10)
ax.set_xlim(-1, 6)
ax.set_ylim(-0.5, 0.5)
ax.set_title("Discrete Token Space\n(Can't interpolate!)")
ax.axhline(0, color='gray', linestyle='--', alpha=0.3)
ax.set_yticks([])
ax.text(2.5, -0.3, "What's between 'cat' and 'dog'? Nothing!", ha='center', fontsize=9)

# Middle: Learned embedding space
ax = axes[1]
# Pretend embeddings arranged nicely for diffusion
emb_positions = {
    'cat': (0, 0), 'dog': (0.5, 0.2), 'mouse': (-0.2, -0.3),
    'run': (1.5, 0), 'walk': (1.3, 0.3), 'fly': (1.7, -0.2),
}
for word, (x, y) in emb_positions.items():
    ax.scatter(x, y, s=100, c='green')
    ax.annotate(word, (x+0.05, y+0.05), fontsize=9)
# Show a noisy point in between
ax.scatter(0.2, 0.1, s=100, c='red', marker='x', linewidths=2)
ax.annotate("noisy\npoint", (0.25, 0.15), fontsize=8, color='red')
ax.set_title("Learned Embedding Space\n(Continuous - can interpolate!)")
ax.set_xlim(-0.5, 2)
ax.set_ylim(-0.5, 0.5)
ax.grid(True, alpha=0.3)

# Right: The advantage
ax = axes[2]
ax.text(0.5, 0.8, "Why Embedding Space?", fontsize=14, ha='center', fontweight='bold', transform=ax.transAxes)
ax.text(0.1, 0.6, "✅ Can add Gaussian noise", fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.5, "✅ Smooth gradients", fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.4, "✅ Standard diffusion math works", fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.25, "⚠️ Must 'round' back to tokens", fontsize=11, transform=ax.transAxes, color='orange')
ax.axis('off')

plt.tight_layout()
plt.show()


#### Classifier Guidance: Steering Generation

One of Diffusion-LM's killer features: **controllable generation**.

**The Problem**: You want to generate text with specific properties (positive sentiment, formal style, about cats, etc.)

**The Solution**: Train a classifier on noisy embeddings, then use its gradients to steer!

**The Math**:
At each denoising step, instead of just following the score, we add the classifier gradient:

$$\nabla_{x_t} \log p(x_{t-1}|x_t, y) \propto \nabla_{x_t} \log p(x_{t-1}|x_t) + \gamma \cdot \nabla_{x_t} \log p(y|x_t)$$

where:
- $\nabla_{x_t} \log p(x_{t-1}|x_t)$ = the diffusion model's denoising direction
- $\nabla_{x_t} \log p(y|x_t)$ = the classifier's "this is class y" direction
- $\gamma$ = guidance strength (how hard to steer)


In [ ]:
# Visualizing classifier guidance

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Setup: imagine 2D embedding space with "positive" and "negative" regions
np.random.seed(42)
positive_center = np.array([1.5, 1.0])
negative_center = np.array([-1.0, -0.5])

# Left: Unguided sampling
ax = axes[0]
# Show trajectory without guidance
trajectory_unguided = [np.array([0, 0])]
for i in range(5):
    next_point = trajectory_unguided[-1] + np.random.randn(2) * 0.3
    trajectory_unguided.append(next_point)
trajectory_unguided = np.array(trajectory_unguided)

ax.scatter(*positive_center, s=300, c='green', marker='*', label='Positive region')
ax.scatter(*negative_center, s=300, c='red', marker='*', label='Negative region')
ax.plot(trajectory_unguided[:, 0], trajectory_unguided[:, 1], 'b-o', linewidth=2, markersize=8, label='Sampling path')
ax.scatter(0, 0, s=100, c='gray', marker='s', zorder=5, label='Start (noise)')
ax.set_title("Without Guidance\n(Random walk)")
ax.legend(loc='lower right', fontsize=8)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2, 2)
ax.grid(True, alpha=0.3)

# Middle: Guided toward positive
ax = axes[1]
trajectory_guided = [np.array([0, 0])]
for i in range(5):
    # Guidance: pull toward positive center
    direction_to_positive = positive_center - trajectory_guided[-1]
    guidance = 0.3 * direction_to_positive / (np.linalg.norm(direction_to_positive) + 0.1)
    next_point = trajectory_guided[-1] + np.random.randn(2) * 0.2 + guidance
    trajectory_guided.append(next_point)
trajectory_guided = np.array(trajectory_guided)

ax.scatter(*positive_center, s=300, c='green', marker='*', label='Positive region')
ax.scatter(*negative_center, s=300, c='red', marker='*', label='Negative region')
ax.plot(trajectory_guided[:, 0], trajectory_guided[:, 1], 'g-o', linewidth=2, markersize=8, label='Guided path')
ax.scatter(0, 0, s=100, c='gray', marker='s', zorder=5, label='Start (noise)')
ax.set_title("With Classifier Guidance\n(Steered toward positive)")
ax.legend(loc='lower right', fontsize=8)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2, 2)
ax.grid(True, alpha=0.3)

# Right: The formula
ax = axes[2]
ax.text(0.5, 0.85, "Guided Score", fontsize=14, ha='center', fontweight='bold', transform=ax.transAxes)
ax.text(0.5, 0.65, r"$\nabla \log p(x_{t-1}|x_t, y)$", fontsize=14, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.45, "=", fontsize=14, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.30, r"$\underbrace{\nabla \log p(x_{t-1}|x_t)}_{\text{diffusion}}$", fontsize=12, ha='center', transform=ax.transAxes)
ax.text(0.5, 0.12, r"$+ \gamma \cdot \underbrace{\nabla \log p(y|x_t)}_{\text{classifier}}$", fontsize=12, ha='center', transform=ax.transAxes)
ax.axis('off')

plt.tight_layout()
plt.show()

print("The classifier gradient 'nudges' the sampling trajectory toward regions")
print("that the classifier associates with the target class!")


In [ ]:
# Clone Diffusion-LM repository (if not already cloned)
import os
import subprocess

DIFFUSION_LM_DIR = "diffusion_lm_repo"

if not os.path.exists(DIFFUSION_LM_DIR):
    print("Cloning Diffusion-LM repository...")
    subprocess.run([
        "git", "clone", 
        "https://github.com/XiangLi1999/Diffusion-LM.git",
        DIFFUSION_LM_DIR
    ], check=True)
    print("Done!")
else:
    print(f"Repository already exists at {DIFFUSION_LM_DIR}")


In [ ]:
# Let's look at the key files in Diffusion-LM
import sys
sys.path.insert(0, DIFFUSION_LM_DIR)

# List the main components
print("Key components in Diffusion-LM:\n")
for root, dirs, files in os.walk(DIFFUSION_LM_DIR):
    # Only show top-level python files
    if root == DIFFUSION_LM_DIR:
        dirs[:] = [d for d in dirs if not d.startswith('.')]  # Skip hidden dirs
        for f in sorted(files):
            if f.endswith('.py'):
                print(f"  {f}")
        print("\n  Subdirectories:")
        for d in sorted(dirs)[:5]:
            print(f"    {d}/")


### Diffusion-LM Architecture Dissection

The key components of Diffusion-LM:

1. **Embedding Layer**: Maps discrete tokens → continuous vectors
2. **Transformer Backbone**: Processes noisy embeddings with timestep conditioning
3. **Rounding Layer**: Maps continuous vectors → discrete tokens
4. **Classifier Guidance**: Controls generation toward desired attributes

Let's examine each:


In [ ]:
# Examine the core model structure
# The main model is in improved-diffusion/improved_diffusion/

import inspect

# Read the key model file
model_file = os.path.join(DIFFUSION_LM_DIR, "improved-diffusion", "improved_diffusion", "transformer_model2.py")

if os.path.exists(model_file):
    with open(model_file, 'r') as f:
        content = f.read()
    
    # Find the main model class
    print("Key classes in transformer_model2.py:\n")
    for line in content.split('\n'):
        if line.strip().startswith('class ') and '(' in line:
            print(f"  {line.strip()}")
else:
    print(f"File not found: {model_file}")
    print("Structure may have changed. Let's find the model files...")
    for root, dirs, files in os.walk(DIFFUSION_LM_DIR):
        for f in files:
            if 'model' in f.lower() and f.endswith('.py'):
                print(f"  {os.path.join(root, f)}")


### Loading Diffusion-LM Pretrained Weights

The authors provide pretrained checkpoints. Let's download and inspect one:


In [ ]:
# Download pretrained checkpoint
# The authors host checkpoints on Google Drive
# We'll use a smaller checkpoint for exploration

import urllib.request
import zipfile

CHECKPOINT_DIR = "diffusion_lm_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Note: You may need to manually download from:
# https://github.com/XiangLi1999/Diffusion-LM#download-checkpoints
# 
# The checkpoints are hosted on Google Drive:
# - e2e-tgt: for E2E dataset
# - roc: for ROCStories dataset

print("Diffusion-LM checkpoints need to be downloaded from Google Drive.")
print("See: https://github.com/XiangLi1999/Diffusion-LM#download-checkpoints")
print()
print("For now, let's understand the architecture by examining the code...")


### Key Insights from Diffusion-LM

**1. The Embedding Trick**
- Use learned embeddings, not frozen ones
- Embeddings are trained jointly with the diffusion model
- This creates a "latent space" optimized for diffusion

**2. The Rounding Problem Solution**
- Soft rounding during training: gradually push embeddings toward token embeddings
- Clamping trick: project back to nearest embedding periodically

**3. Classifier Guidance**
- Train a classifier on noisy embeddings
- At generation time, use classifier gradient to steer toward desired properties
- Example: generate text with positive sentiment

**4. Limitations**
- Slow sampling (many denoising steps)
- Rounding errors accumulate
- Not as fluent as autoregressive models


---

### DiffuSeq: Conditioning for Seq2Seq (Deep Dive)

**Paper**: "DiffuSeq: Sequence to Sequence Text Generation with Diffusion Models" (Gong et al., 2023)

#### Why DiffuSeq Matters

Diffusion-LM showed diffusion can generate text. But real NLP tasks need **conditioning**:
- Translation: English → French
- Summarization: Long article → Short summary
- QA: Question → Answer

DiffuSeq answers: **How does diffusion replace the autoregressive decoder in seq2seq?**

#### The Conditioning Problem

In autoregressive seq2seq (like T5):
```
Encoder: "The cat sat" → hidden states
Decoder: Generates "Le chat..." token by token, attending to encoder
```

In diffusion, we generate all at once. How do we condition on the source?

#### The DiffuSeq Solution: Partial Noising

```
Input:  [source tokens] | [target tokens]
        [The] [cat] [sat] | [Le] [chat] [assis]
                          
Forward: Only noise the TARGET portion!
        [The] [cat] [sat] | [MASK] [MASK] [MASK]  (t=T)
        
Reverse: Denoise target while attending to source
        [The] [cat] [sat] | [Le] [chat] [assis]   (t=0)
```

The source tokens stay clean throughout - the model can always "look" at them!

#### Why This Works

1. **Self-attention across concatenated sequence**: Target attends to source
2. **Source provides context**: Even at high noise, model knows what to generate about
3. **No separate encoder needed**: Single transformer does everything
4. **Natural for seq2seq**: Directly maps input→output


In [ ]:
# Visualizing DiffuSeq's partial noising strategy

fig, axes = plt.subplots(2, 4, figsize=(16, 6))

source = ["The", "cat", "sat"]
target_clean = ["Le", "chat", "assis"]

# Different timesteps showing the denoising
timesteps = [100, 75, 50, 0]
target_at_t = [
    ["[M]", "[M]", "[M]"],
    ["[M]", "chat", "[M]"],
    ["Le", "chat", "[M]"],
    ["Le", "chat", "assis"],
]

for i, (t, tgt) in enumerate(zip(timesteps, target_at_t)):
    ax = axes[0, i]
    # Draw source (always clean)
    for j, tok in enumerate(source):
        ax.add_patch(plt.Rectangle((j*1.2, 0.5), 1, 0.4, color='lightgreen', ec='green', lw=2))
        ax.text(j*1.2 + 0.5, 0.7, tok, ha='center', va='center', fontsize=10)
    # Draw target
    for j, tok in enumerate(tgt):
        color = 'lightgray' if tok == "[M]" else 'lightblue'
        ax.add_patch(plt.Rectangle((j*1.2 + 4, 0.5), 1, 0.4, color=color, ec='blue', lw=2))
        ax.text(j*1.2 + 4.5, 0.7, tok, ha='center', va='center', fontsize=10)
    ax.set_xlim(-0.5, 8)
    ax.set_ylim(0, 1.5)
    ax.set_title(f"t = {t}" + (" (noise)" if t == 100 else " (clean)" if t == 0 else ""))
    ax.axis('off')
    # Labels
    if i == 0:
        ax.text(1.5, 1.2, "Source\n(always clean)", ha='center', fontsize=9, color='green')
        ax.text(5.5, 1.2, "Target\n(denoising)", ha='center', fontsize=9, color='blue')

# Bottom row: attention pattern
ax = axes[1, 0]
ax.text(0.5, 0.5, "Self-Attention Pattern", ha='center', va='center', fontsize=12, fontweight='bold', transform=ax.transAxes)
ax.axis('off')

ax = axes[1, 1]
# Show attention matrix
attn = np.zeros((6, 6))
attn[:3, :3] = 0.3  # Source attends to source
attn[3:, :3] = 0.8  # Target attends to source (key!)
attn[3:, 3:] = 0.5  # Target attends to target
ax.imshow(attn, cmap='Blues', vmin=0, vmax=1)
ax.set_xticks(range(6))
ax.set_xticklabels(['The', 'cat', 'sat', 'Le', 'chat', 'assis'], fontsize=8)
ax.set_yticks(range(6))
ax.set_yticklabels(['The', 'cat', 'sat', 'Le', 'chat', 'assis'], fontsize=8)
ax.set_title("Attention weights", fontsize=10)
ax.set_xlabel("Keys (what we attend TO)")
ax.set_ylabel("Queries (who is attending)")

ax = axes[1, 2]
ax.text(0.5, 0.7, "Key Insight:", ha='center', fontsize=11, fontweight='bold', transform=ax.transAxes)
ax.text(0.5, 0.45, "Target tokens attend to\nsource tokens even when\ntarget is fully masked!", ha='center', fontsize=10, transform=ax.transAxes)
ax.axis('off')

ax = axes[1, 3]
ax.text(0.5, 0.8, "Advantages:", ha='center', fontsize=11, fontweight='bold', transform=ax.transAxes)
ax.text(0.1, 0.55, "✅ Single unified model", fontsize=10, transform=ax.transAxes)
ax.text(0.1, 0.40, "✅ No encoder-decoder split", fontsize=10, transform=ax.transAxes)
ax.text(0.1, 0.25, "✅ Easy to implement", fontsize=10, transform=ax.transAxes)
ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# DiffuSeq conditioning concept (simplified)

def diffuseq_forward(source_tokens, target_tokens, t, alpha_bar, mask_id):
    """
    Only apply noise to target, keep source fixed.
    """
    # Source stays clean
    source_noisy = source_tokens.clone()
    
    # Target gets noised
    target_noisy, mask = discrete_forward_diffusion(
        target_tokens.unsqueeze(0), 
        torch.tensor([t]), 
        alpha_bar, 
        mask_id
    )
    
    # Concatenate
    combined = torch.cat([source_noisy.unsqueeze(0), target_noisy], dim=1)
    return combined.squeeze(0)

# Demo
source = torch.tensor([word2id["the"], word2id["cat"], word2id["sat"]])  # Source: "the cat sat"
target = torch.tensor([word2id["on"], word2id["the"], word2id["mat"]])    # Target: "on the mat"

print("DiffuSeq-style conditioning:\n")
print(f"Source (clean): {' '.join([id2word[i.item()] for i in source])}")
print(f"Target (clean): {' '.join([id2word[i.item()] for i in target])}")
print()

for t in [0, 50, 99]:
    combined = diffuseq_forward(source, target, t, alpha_bar_discrete, MASK_ID)
    src_part = ' '.join([id2word[i.item()] for i in combined[:3]])
    tgt_part = ' '.join([id2word[i.item()] for i in combined[3:]])
    print(f"t={t:2d}: [{src_part}] | [{tgt_part}]")


---

### MMDiT: Modern Transformer Diffusion (Deep Dive)

**Architecture**: Multimodal Diffusion Transformer (MMDiT) - the architecture behind Stable Diffusion 3

#### The Evolution: How MMDiT Builds on Previous Work

| Model | Year | Key Innovation | Limitation Solved |
|-------|------|----------------|-------------------|
| **Diffusion-LM** | 2022 | Embedding-space diffusion | Discrete→continuous |
| **DiffuSeq** | 2023 | Partial noising for conditioning | Seq2seq tasks |
| **MMDiT** | 2024 | Pure transformer, AdaLN, latent space | Scalability, multimodal |

#### Why MMDiT is the Future

**1. Transformers Replace UNets**
- UNets were designed for images (locality bias)
- Transformers scale better and are more flexible
- Same architecture works for text, images, audio, video

**2. Latent Space Diffusion**
- Don't diffuse raw tokens/pixels - too expensive
- First compress to latent space (VAE/tokenizer)
- Diffuse in compressed space
- Decode final output

**3. AdaLN: Elegant Timestep Conditioning**
Instead of awkward timestep addition, modulate the LayerNorm:
$$\text{AdaLN}(x, t) = \gamma(t) \cdot \text{LayerNorm}(x) + \beta(t)$$

**4. Classifier-Free Guidance (CFG)**
- No separate classifier needed
- Train with condition dropout
- At inference: $\epsilon = \epsilon_{\text{uncond}} + \gamma(\epsilon_{\text{cond}} - \epsilon_{\text{uncond}})$

<!-- 
IMAGE REQUEST: MMDiT architecture diagram
URL: Stable Diffusion 3 paper or OpenMMDiT repo
-->


In [ ]:
# Visualizing the evolution: Diffusion-LM → DiffuSeq → MMDiT

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Diffusion-LM
ax = axes[0]
ax.set_title("Diffusion-LM (2022)", fontsize=12, fontweight='bold')
boxes = [
    ("Tokens", 0.1, 0.8, 'lightblue'),
    ("Learned\nEmbeddings", 0.1, 0.6, 'lightgreen'),
    ("Gaussian\nDiffusion", 0.1, 0.4, 'lightyellow'),
    ("Transformer\nDenoiser", 0.1, 0.2, 'lightcoral'),
    ("Round to\nTokens", 0.1, 0.0, 'lightblue'),
]
for label, x, y, color in boxes:
    ax.add_patch(plt.Rectangle((x, y), 0.8, 0.15, color=color, ec='black'))
    ax.text(0.5, y + 0.075, label, ha='center', va='center', fontsize=9)
    if y > 0:
        ax.annotate("", xy=(0.5, y), xytext=(0.5, y + 0.02),
                    arrowprops=dict(arrowstyle="->", color='gray'))
ax.text(0.5, -0.15, "Unconditional\n+ Classifier Guidance", ha='center', fontsize=9, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(-0.2, 1)
ax.axis('off')

# DiffuSeq
ax = axes[1]
ax.set_title("DiffuSeq (2023)", fontsize=12, fontweight='bold')
# Source + Target concatenation
ax.add_patch(plt.Rectangle((0.05, 0.8), 0.4, 0.15, color='lightgreen', ec='green', lw=2))
ax.text(0.25, 0.875, "Source", ha='center', va='center', fontsize=9)
ax.add_patch(plt.Rectangle((0.55, 0.8), 0.4, 0.15, color='lightblue', ec='blue', lw=2))
ax.text(0.75, 0.875, "Target", ha='center', va='center', fontsize=9)
ax.add_patch(plt.Rectangle((0.1, 0.5), 0.8, 0.2, color='lightyellow', ec='black'))
ax.text(0.5, 0.6, "Partial Noising\n(only target)", ha='center', va='center', fontsize=9)
ax.add_patch(plt.Rectangle((0.1, 0.2), 0.8, 0.2, color='lightcoral', ec='black'))
ax.text(0.5, 0.3, "Transformer\n(bidirectional)", ha='center', va='center', fontsize=9)
ax.annotate("", xy=(0.5, 0.5), xytext=(0.5, 0.75), arrowprops=dict(arrowstyle="->", color='gray'))
ax.annotate("", xy=(0.5, 0.2), xytext=(0.5, 0.45), arrowprops=dict(arrowstyle="->", color='gray'))
ax.text(0.5, -0.05, "Conditional\n(seq2seq tasks)", ha='center', fontsize=9, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(-0.2, 1)
ax.axis('off')

# MMDiT
ax = axes[2]
ax.set_title("MMDiT (2024)", fontsize=12, fontweight='bold')
ax.add_patch(plt.Rectangle((0.1, 0.75), 0.35, 0.15, color='lightblue', ec='blue'))
ax.text(0.275, 0.825, "Noisy\nLatent", ha='center', va='center', fontsize=8)
ax.add_patch(plt.Rectangle((0.55, 0.75), 0.35, 0.15, color='lightgreen', ec='green'))
ax.text(0.725, 0.825, "Condition\nLatent", ha='center', va='center', fontsize=8)
# DiT blocks
for i, y in enumerate([0.55, 0.35, 0.15]):
    ax.add_patch(plt.Rectangle((0.1, y), 0.8, 0.12, color='lightcoral', ec='black'))
    ax.text(0.5, y + 0.06, f"DiT Block (AdaLN + Attn)", ha='center', va='center', fontsize=8)
ax.add_patch(plt.Rectangle((0.1, -0.05), 0.8, 0.12, color='lightyellow', ec='black'))
ax.text(0.5, 0.01, "Output Projection", ha='center', va='center', fontsize=8)
ax.text(0.92, 0.4, "+t", fontsize=10, color='purple', fontweight='bold')
ax.annotate("", xy=(0.5, 0.55), xytext=(0.5, 0.7), arrowprops=dict(arrowstyle="->", color='gray'))
ax.text(0.5, -0.18, "Multimodal + Scalable\n(SOTA)", ha='center', fontsize=9, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(-0.25, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

print("Evolution: simpler → more general → more scalable")


#### AdaLN: Why Modulate Instead of Add?

**The Old Way (Diffusion-LM style)**:
```python
h = h + timestep_embedding  # Add timestep to hidden states
```
Problem: timestep just gets "mixed in" - network must learn to disentangle

**The New Way (AdaLN)**:
```python
h = LayerNorm(h) * (1 + scale(t)) + shift(t)  # MODULATE the normalization
```

**Why is modulation better?**

Think of LayerNorm as "resetting" the scale of activations. AdaLN says:
- "At timestep $t$, activations should be scaled by $\gamma(t)$"
- "At timestep $t$, activations should be shifted by $\beta(t)$"

It's like telling each layer "how much to react" based on the noise level.

**Intuition**: At high noise ($t$ large), you want big, coarse changes. At low noise ($t$ small), you want fine, subtle changes. AdaLN can express this naturally through scale!


In [ ]:
# | export
class AdaLN(nn.Module):
    """
    Adaptive Layer Normalization - the key timestep conditioning mechanism in modern diffusion.
    
    Instead of adding timestep embedding, we use it to modulate the LayerNorm:
    output = (x - mean) / std * (1 + scale) + shift
    
    where scale and shift come from the timestep embedding.
    """
    def __init__(self, d_model, time_dim):
        super().__init__()
        self.norm = nn.LayerNorm(d_model, elementwise_affine=False)
        self.modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim, 2 * d_model)
        )
    
    def forward(self, x, time_emb):
        """
        x: [batch, seq_len, d_model]
        time_emb: [batch, time_dim]
        """
        # Get scale and shift from time embedding
        scale_shift = self.modulation(time_emb)  # [batch, 2*d_model]
        scale, shift = scale_shift.chunk(2, dim=-1)
        
        # Normalize then modulate
        x = self.norm(x)
        x = x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)
        return x

# Demo
adaln = AdaLN(d_model=64, time_dim=64)
x = torch.randn(2, 10, 64)  # [batch=2, seq_len=10, d_model=64]
t_emb = torch.randn(2, 64)  # [batch=2, time_dim=64]
out = adaln(x, t_emb)
print(f"AdaLN: input {x.shape} → output {out.shape}")
print("Timestep modulates the normalization scale and shift!")


---

## Building a SOTA-Shaped Text Diffusion Model

Now let's build a proper, scalable text diffusion model. We'll:

1. Use discrete diffusion (more practical for text)
2. Implement AdaLN for timestep conditioning
3. Add self-conditioning (a key trick)
4. Design for ~100-200M parameters
5. Train on a real dataset

### Architecture Design

Target specs for training on 2x T4 GPUs:
- **d_model**: 768
- **n_heads**: 12  
- **n_layers**: 12
- **vocab_size**: ~30K (using a pretrained tokenizer)
- **max_seq_len**: 256
- **Parameters**: ~125M


In [ ]:
# | export
class DiTBlock(nn.Module):
    """
    A Diffusion Transformer block with AdaLN conditioning.
    """
    def __init__(self, d_model, n_heads, dropout=0.0):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model),
        )
        self.adaln1 = AdaLN(d_model, d_model)
        self.adaln2 = AdaLN(d_model, d_model)
        
    def forward(self, x, t_emb):
        # Pre-norm with AdaLN
        x_norm = self.adaln1(x, t_emb)
        x = x + self.attn(x_norm, x_norm, x_norm, need_weights=False)[0]
        
        x_norm = self.adaln2(x, t_emb)
        x = x + self.mlp(x_norm)
        return x


In [ ]:
# | export
class TextDiT(nn.Module):
    """
    Text Diffusion Transformer - a SOTA-shaped discrete text diffusion model.
    
    Features:
    - Discrete diffusion (mask-based)
    - AdaLN timestep conditioning
    - Self-conditioning (optional)
    - Bidirectional attention (can see all positions)
    """
    def __init__(
        self,
        vocab_size,
        d_model=768,
        n_heads=12,
        n_layers=12,
        max_seq_len=256,
        dropout=0.0,
        self_condition=True,
    ):
        super().__init__()
        self.d_model = d_model
        self.self_condition = self_condition
        
        # Token embedding
        self.token_emb = nn.Embedding(vocab_size, d_model)
        
        # Position embedding
        self.pos_emb = nn.Embedding(max_seq_len, d_model)
        
        # Timestep embedding
        self.time_emb = nn.Sequential(
            SinusoidalPosEmb(d_model),
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model),
        )
        
        # Self-conditioning: embed previous prediction
        if self_condition:
            self.self_cond_proj = nn.Linear(vocab_size, d_model)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            DiTBlock(d_model, n_heads, dropout)
            for _ in range(n_layers)
        ])
        
        # Final norm and output
        self.final_norm = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size)
        
        # Initialize output projection to small values
        nn.init.zeros_(self.output_proj.weight)
        nn.init.zeros_(self.output_proj.bias)
        
    def forward(self, x, t, self_cond=None):
        """
        Args:
            x: Token ids [batch, seq_len]
            t: Timesteps [batch]
            self_cond: Previous logits prediction [batch, seq_len, vocab_size] (optional)
        Returns:
            logits: [batch, seq_len, vocab_size]
        """
        batch_size, seq_len = x.shape
        device = x.device
        
        # Embeddings
        positions = torch.arange(seq_len, device=device)
        h = self.token_emb(x) + self.pos_emb(positions)
        
        # Add self-conditioning if provided
        if self.self_condition and self_cond is not None:
            h = h + self.self_cond_proj(self_cond)
        
        # Timestep embedding
        t_emb = self.time_emb(t.float())
        
        # Transformer blocks
        for block in self.blocks:
            h = block(h, t_emb)
        
        # Output
        h = self.final_norm(h)
        logits = self.output_proj(h)
        
        return logits


In [ ]:
# Create a SOTA-shaped model and check parameter count
# We'll use a tokenizer from transformers for real vocab

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
real_vocab_size = len(tokenizer)

# Create model at target scale
sota_model = TextDiT(
    vocab_size=real_vocab_size,
    d_model=768,
    n_heads=12,
    n_layers=12,
    max_seq_len=256,
    self_condition=True,
)

n_params = sum(p.numel() for p in sota_model.parameters())
print(f"Vocabulary size: {real_vocab_size:,}")
print(f"Model parameters: {n_params:,}")
print(f"Model size: ~{n_params / 1e6:.1f}M parameters")
print()

# Breakdown by component
print("Parameter breakdown:")
for name, module in sota_model.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"  {name}: {params:,} ({params/n_params*100:.1f}%)")


### Dataset: TinyStories

For training, we'll use **TinyStories** - a dataset of simple stories generated by GPT-3.5/4, designed for training small language models.

**Why TinyStories?**
- Clean, coherent text
- Simple patterns that small models can learn
- Manageable size for quick experiments
- Good for seeing if diffusion can generate coherent text


In [ ]:
# Load TinyStories dataset
from datasets import load_dataset

# Load a subset for faster experimentation
dataset = load_dataset("roneneldan/TinyStories", split="train[:50000]")
print(f"Dataset size: {len(dataset):,} stories")
print()

# Preview a sample
print("Sample story:")
print("-" * 40)
print(dataset[0]["text"][:500])


In [ ]:
# Tokenize dataset
MAX_LEN = 128  # Shorter for faster training

def tokenize_function(examples):
    # Tokenize and truncate/pad
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt"
    )

# Process in batches
tokenized = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing"
)

print(f"Tokenized {len(tokenized):,} examples")
print(f"Each example: {MAX_LEN} tokens")


In [ ]:
# Create DataLoader
from torch.utils.data import DataLoader

tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

train_loader = DataLoader(
    tokenized,
    batch_size=32,  # Adjust based on GPU memory
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

# Test batch
batch = next(iter(train_loader))
print(f"Batch shape: {batch['input_ids'].shape}")


### Training with Weights & Biases

We'll use WandB to log:
- Loss curves (overall and by timestep)
- Sample generations during training
- Gradient norms
- Learning rate


In [ ]:
import wandb
from tqdm import tqdm

# Initialize WandB
wandb.init(
    project="text-diffusion",
    config={
        "d_model": 768,
        "n_heads": 12,
        "n_layers": 12,
        "max_seq_len": MAX_LEN,
        "batch_size": 32,
        "lr": 1e-4,
        "num_steps": 1000,
        "dataset": "tinystories",
        "self_condition": True,
    }
)


In [ ]:
# | export
def train_step(model, batch, optimizer, num_steps, alpha_bar, mask_token_id, device, self_cond_prob=0.5):
    """
    Single training step with self-conditioning.
    """
    model.train()
    
    x_0 = batch["input_ids"].to(device)
    batch_size = x_0.shape[0]
    
    # Sample random timesteps
    t = torch.randint(0, num_steps, (batch_size,), device=device)
    
    # Forward diffusion
    x_t, mask = discrete_forward_diffusion(x_0, t, alpha_bar.to(device), mask_token_id)
    
    # Self-conditioning: 50% of time, use previous prediction
    self_cond = None
    if model.self_condition and torch.rand(1).item() < self_cond_prob:
        with torch.no_grad():
            self_cond = model(x_t, t, self_cond=None)
            self_cond = F.softmax(self_cond, dim=-1)
    
    # Forward pass
    logits = model(x_t, t, self_cond=self_cond)
    
    # Loss: cross-entropy on all positions
    # (For simplicity we compute on all, but weight masked more)
    loss = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        x_0.view(-1),
        reduction='none'
    )
    
    # Weight masked positions more heavily
    mask_weight = mask.float() * 2.0 + 1.0  # masked: 3x, unmasked: 1x
    loss = (loss.view(batch_size, -1) * mask_weight).mean()
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    
    # Gradient clipping
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    optimizer.step()
    
    return loss.item(), grad_norm.item()


In [ ]:
# Prepare for training
# Use smaller model for demo (full model needs more GPU memory)
demo_model = TextDiT(
    vocab_size=real_vocab_size,
    d_model=256,
    n_heads=4,
    n_layers=4,
    max_seq_len=MAX_LEN,
    self_condition=True,
).to(device)

demo_params = sum(p.numel() for p in demo_model.parameters())
print(f"Demo model: {demo_params:,} parameters (~{demo_params/1e6:.1f}M)")

# Optimizer
optimizer = torch.optim.AdamW(demo_model.parameters(), lr=1e-4, weight_decay=0.01)

# Noise schedule
NUM_TRAIN_STEPS = 1000
train_alpha_bar = cosine_schedule(NUM_TRAIN_STEPS)

# MASK token for GPT-2 tokenizer (we'll add one)
MASK_TOKEN_ID = tokenizer.encode("[MASK]", add_special_tokens=False)[0] if "[MASK]" in tokenizer.vocab else tokenizer.eos_token_id
print(f"Using MASK token ID: {MASK_TOKEN_ID}")


In [ ]:
# Training loop
NUM_EPOCHS = 2  # Increase for better results
LOG_INTERVAL = 100
SAMPLE_INTERVAL = 500

global_step = 0
losses_list = []

for epoch in range(NUM_EPOCHS):
    epoch_losses = []
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for batch in pbar:
        loss, grad_norm = train_step(
            demo_model, batch, optimizer, 
            NUM_TRAIN_STEPS, train_alpha_bar, 
            MASK_TOKEN_ID, device
        )
        
        epoch_losses.append(loss)
        losses_list.append(loss)
        global_step += 1
        
        # Update progress bar
        pbar.set_postfix({"loss": f"{loss:.4f}", "grad_norm": f"{grad_norm:.2f}"})
        
        # Log to wandb
        if global_step % LOG_INTERVAL == 0:
            wandb.log({
                "loss": loss,
                "grad_norm": grad_norm,
                "epoch": epoch,
                "step": global_step,
            })
        
        # Generate samples periodically
        if global_step % SAMPLE_INTERVAL == 0:
            demo_model.eval()
            with torch.no_grad():
                # Generate from all masks
                gen_input = torch.full((1, MAX_LEN), MASK_TOKEN_ID, device=device)
                gen_t = torch.tensor([NUM_TRAIN_STEPS-1], device=device)
                gen_output = demo_model(gen_input, gen_t)
                gen_tokens = gen_output.argmax(dim=-1)
                gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
                print(f"\n[Step {global_step}] Sample: {gen_text[:100]}...")
            demo_model.train()
    
    print(f"Epoch {epoch+1} avg loss: {sum(epoch_losses)/len(epoch_losses):.4f}")

wandb.finish()
print("Training complete!")


---

## Production Diagnostics: What's Different from Transformers?

When training text diffusion models, there are specific things to monitor that differ from standard transformers.


In [ ]:
### Key Differences: Transformer vs Diffusion Training

| Aspect | **Autoregressive Transformer** | **Diffusion Model** |
|--------|-------------------------------|---------------------|
| **Loss** | Single loss (next-token CE) | Loss varies by timestep! Early (high $t$) is easy, late (low $t$) is hard |
| **What to monitor** | Overall loss curve | **Loss per timestep bucket** - crucial for debugging |
| **Gradients** | Single objective | Mixed from different timesteps - some dominate |
| **Gradient fix** | Standard clipping | Loss weighting or importance sampling by $t$ |
| **Quality metrics** | Perplexity (well-defined) | No clean PPL - use BLEU, MAUVE, diversity |
| **EMA** | Helps | **Almost required** for stable sampling |

#### Why Timestep-Specific Monitoring Matters

In diffusion:
- **High $t$ (noisy)**: Easy to learn - just predict "it's noise"
- **Low $t$ (clean)**: Hard - need to predict fine details
- **If loss only drops at high $t$**: Model learned nothing useful!

This is very different from transformers where a single loss tells the whole story.


In [ ]:
# | export
def compute_loss_by_timestep(model, dataloader, num_steps, alpha_bar, mask_id, device, n_buckets=10):
    """
    Compute loss broken down by timestep buckets.
    Useful for understanding where the model struggles.
    """
    model.eval()
    bucket_size = num_steps // n_buckets
    bucket_losses = [[] for _ in range(n_buckets)]
    
    with torch.no_grad():
        for batch in dataloader:
            x_0 = batch["input_ids"].to(device)
            
            # Test each bucket
            for bucket_idx in range(n_buckets):
                t_low = bucket_idx * bucket_size
                t_high = (bucket_idx + 1) * bucket_size
                t = torch.randint(t_low, t_high, (x_0.shape[0],), device=device)
                
                x_t, mask = discrete_forward_diffusion(x_0, t, alpha_bar.to(device), mask_id)
                logits = model(x_t, t)
                
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
                    x_0.view(-1)
                )
                bucket_losses[bucket_idx].append(loss.item())
    
    return [sum(b)/len(b) if b else 0 for b in bucket_losses]

# Demo: analyze loss by timestep (using small sample)
small_loader = DataLoader(tokenized.select(range(100)), batch_size=10)
bucket_losses = compute_loss_by_timestep(
    demo_model, small_loader, NUM_TRAIN_STEPS, 
    train_alpha_bar, MASK_TOKEN_ID, device, n_buckets=10
)

plt.figure(figsize=(10, 5))
plt.bar(range(10), bucket_losses)
plt.xlabel("Timestep bucket (0=clean, 9=noisy)")
plt.ylabel("Loss")
plt.title("Loss by Timestep: Where Does the Model Struggle?")
plt.xticks(range(10), [f"{i*100}-{(i+1)*100}" for i in range(10)])
plt.grid(True, alpha=0.3)
plt.show()

print("Higher loss at low timesteps = model struggles with fine details")
print("Higher loss at high timesteps = model struggles with global structure")


### Diffusion-Specific Tricks

These are techniques that are unique or especially important for diffusion models:


In [ ]:
# | export
class EMAModel:
    """
    Exponential Moving Average of model weights.
    Critical for stable diffusion sampling.
    """
    def __init__(self, model, decay=0.9999):
        self.decay = decay
        self.shadow = {}
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()
    
    def update(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = (
                    self.decay * self.shadow[name] + 
                    (1 - self.decay) * param.data
                )
    
    def apply(self, model):
        """Apply EMA weights to model (for sampling)."""
        for name, param in model.named_parameters():
            if param.requires_grad:
                param.data.copy_(self.shadow[name])

print("1. EMA (Exponential Moving Average)")
print("   - Keeps smoothed version of weights")
print("   - Use EMA weights for sampling, not training weights")
print("   - Decay typically 0.9999")
print()


In [ ]:
print("2. SELF-CONDITIONING")
print("   - Feed previous prediction back as input")
print("   - During training: 50% use self-cond, 50% don't")
print("   - During sampling: always use self-cond")
print("   - Improves sample quality significantly")
print()

print("3. NOISE SCHEDULE TUNING")
print("   - Cosine schedule > linear schedule")
print("   - May need to adjust for text vs images")
print("   - More aggressive schedule for shorter sequences")
print()

print("4. CLASSIFIER-FREE GUIDANCE")
print("   - Drop conditioning during training with some probability")
print("   - At inference, interpolate between conditional and unconditional")
print("   - Controls diversity vs fidelity tradeoff")
print()

print("5. TIMESTEP IMPORTANCE SAMPLING")
print("   - Some timesteps are more informative")
print("   - Sample harder timesteps more often")
print("   - Or use loss-weighted sampling")


---

## Summary: What We've Learned

### The Core Idea
Diffusion = **iteratively refine from noise to data**. For text:
- Either diffuse in embedding space (Diffusion-LM style)
- Or use discrete masking (modern MDLM/SEDD style)

### The Key Components
1. **Forward process**: Corrupt data with noise/masks
2. **Denoising network**: Predict clean data from noisy
3. **Noise schedule**: Control corruption rate (cosine works well)
4. **Reverse process**: Iteratively denoise to generate

### Evolution of Text Diffusion
1. **Diffusion-LM** (2022): Continuous embeddings, classifier guidance
2. **DiffuSeq** (2023): Seq2seq conditioning
3. **MDLM/SEDD** (2024): Discrete diffusion, better discrete math
4. **MMDiT** (2024+): Transformer-native diffusion

### When to Use Text Diffusion vs Autoregressive
| Use Case | Best Approach |
|----------|--------------|
| General text generation | Autoregressive (GPT) |
| Controllable generation | Diffusion (guidance) |
| Infilling / editing | Diffusion |
| Speed-critical | Autoregressive |
| Parallel generation | Diffusion |

### Resources for Further Learning
- **Papers**: "Diffusion-LM", "MDLM", "Score Entropy Discrete Diffusion"
- **Repos**: [Diffusion-LM](https://github.com/XiangLi1999/Diffusion-LM), [DiffuSeq](https://github.com/Shark-NLP/DiffuSeq)
- **Active area**: Watch for new discrete diffusion methods!


## The Text Problem: Discrete vs Continuous

Here's the fundamental challenge: **text is discrete**.

In images, a pixel can smoothly transition: `[0.5, 0.5, 0.5]` → `[0.51, 0.49, 0.52]`

In text, tokens are integers: "cat" is token `3784`. What's `3784 + 0.3`?
vide way s and 

## The Text Problem: Discrete vs Continuous

Here's the fundamental challenge: **text is discrete**.

In images, a pixel can smoothly transition: `[0.5, 0.5, 0.5]` → `[0.51, 0.49, 0.52]`

In text, tokens are integers: "cat" is token `3784`. What's `3784 + 0.3`?
